In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

import sys
from dgnutils import *

sys.path.append('/Users/nish/Local/Programming/convertkit_wrapper')
from modules.ck_utils import *

session = requests.session()

import matplotlib.pyplot as plt
import reverse_geocoder as rg
import iso3166

8/21/19 dgnutils update loaded!


In [2]:
try: conn.close()
except: pass
conn, cursor = connect()

In [ ]:
# Refresher for "Could not confirm type of CK_Api.ALL_SUBS, CK_Api.SEQ_SUBS, False"
from modules.ck_utils import *

#### Quick tests

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE id = %s', 137633175); cursor.fetchall()

In [ ]:
# Update purchases where the purchase_id = the given id from the webhook
cursor.e('SELECT * FROM purchase_data WHERE purchaser_email_address LIKE "%wend%"')

## Get Unsubscribers

### Loop

In [ ]:
updated_unsubs = []
day_date = date(2019,4,20) # Start Date

while day_date <= date.today():
    
    day = day_date.strftime("%Y-%m-%d")
    unsubs = get_new_unsub_for_day(day, exclude_updates=True)
    
    if unsubs:
        time.sleep(1) # Slow down API calls
        print(f'updating {len(unsubs)} unsubscribers for {day}')
    
        for unsub in unsubs:
            updated_unsubs.append(unsub['id']) #Record updates
            update_unsub_with_day(unsub['id'], day) #Make update
        
    day_date += timedelta(days=1) # Increment day

In [7]:
def get_ck_session():
        """Handle the initial page load and session creation
        Returns an authenticated session"""

        session_requests = requests.session()
        login_url = "https://app.convertkit.com/users/login"

        result = session_requests.get(login_url)
        login_tree = bs4.BeautifulSoup(result.text, 'html.parser'); login_tree
        if not result.ok: raise ConnectionError(result.status_code)

        payload = {
                "user[email]": os.environ['CK_USER'],
                "user[password]": os.environ['CK_PASSWORD'],
                "authenticity_token": login_tree.find("input", {"name":"authenticity_token"})['value']
        };

        login_result = session_requests.post(login_url, data = payload, headers = dict(referer=login_url));
        if not login_result.ok: raise ConnectionError(login_result.status_code)
        return session_requests

In [ ]:
updated_unsubs[-3:]

In [4]:
s = Session('app.convertkit.com/subscribers/137633017', persist_cookies=True, connections=20)

In [52]:
Out[42].status_code

200

### Fix unsubscribe differences

In [38]:
payload = {
                "user[email]": os.environ['CK_USER'],
                "user[password]": os.environ['CK_PASSWORD'],
                "authenticity_token": login_tree.find("input", {"name":"authenticity_token"})['value']
        };

NameError: name 'login_tree' is not defined

In [ ]:
# Shouldn't have any of these after the 6/3/19 update to the unsubscribe lambda

In [ ]:
subscribers_unsub = get_all_sub_data(columns=[], sort_field='cancelled_at')
len(subscribers_unsub), subscribers_unsub[0]

In [ ]:
%

In [70]:
def test():
    return 2+2
await test()

TypeError: object int can't be used in 'await' expression

In [ ]:
# go through each unsub and see if rds does not match
unsub_updates = []
for unsub in subscribers_unsub:
    if unsub[0] in sub_dict and sub_dict[unsub[0]]['state']!='cancelled':
        unsub_updates.append([unsub[13], unsub[0]])
len(unsub_updates), unsub_updates[:3]

In [73]:
scrape_email_list(137633017)

[{'delivered': True,
  'opened': False,
  'clicked': False,
  'bounced': False,
  'failed': False,
  'deliver_date': datetime.datetime(2019, 9, 17, 10, 51, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  'open_date': None,
  'click_date': None,
  'email_id': '2863522',
  'email_name': '[Might Could Essay] When Trying Too Hard Backfires'},
 {'delivered': True,
  'opened': False,
  'clicked': False,
  'bounced': False,
  'failed': False,
  'deliver_date': datetime.datetime(2019, 9, 10, 13, 48, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  'open_date': None,
  'click_date': None,
  'email_id': '2838431',
  'email_name': '[Might Could Essay] Inspiration vs. Imitation: How to Copy as an Artist'},
 {'delivered': True,
  'opened': True,
  'clicked': False,
  'bounced': False,
  'failed': False,
  'deliver_date': datetime.datetime(2019, 9, 3, 12, 49, tzinfo=<DstTzInfo 'US/Eastern' EDT-1 day, 20:00:00 DST>),
  'open_date': datetime.datetime(2019, 9, 3, 12, 58, tzinfo=<

In [ ]:
# Update RDS
# cursor.executemany('UPDATE subscriber_data SET unsubscribe_date=%s,state="cancelled" WHERE id=%s', unsub_updates)

In [ ]:
conn.commit()

### All CK unsubscribers

In [ ]:
from modules.ck_utils import *
cancellers = get_api_data(CK_Api.ALL_SUBS, None, include_unsubscribers=True)

all_rds_list = cursor.e('SELECT * FROM subscriber_data')
all_rds_dict = {a['id']: a for a in all_rds_list}

missing_cancellers = [c for c in cancellers if c['id'] not in all_rds_dict]

fixed_cancellers = [{k:v for k,v in c.items() if k not in ['last_name']} for c in missing_cancellers]

cursor.dict_insert(fixed_cancellers, 'subscriber_data')

# conn.commit()

## Webhook Work

In [ ]:
# Subscriber id
url = f'https://api.convertkit.com/v3/subscribers'
data = json.dumps({'api_secret': API_SECRET, 'email_address': 'dnishiyama+test@gmail.com'}); data #parse.quote_plus(
resp = requests.get(url=url, data=data, headers={'content-type': 'application/json'})
if resp.status_code != 200: 
    print(r.status_code) 
    print(r.text)
json_resp = json.loads(resp.text); json_resp
sub_item = [sub['id'] for sub in json_resp['subscribers']]
sub_id = sub_item[0] if sub_item else None
sub_id

In [ ]:
# Tag id
# Purchased: Sketchbook to Style --- 291275
# StSBC Launch - EVG --- 899142
url = f'https://api.convertkit.com/v3/tags?api_key={API_KEY}'
data = json.dumps({'api_key': API_KEY}); data
resp = requests.get(url=url, data=data, headers={'content-type': 'application/json'}); resp
if resp.status_code != 200: 
    print(r.status_code) 
    print(r.text)
json_resp = json.loads(resp.text); json_resp
tag_item = [tag['id'] for tag in json_resp['tags'] if tag['name'] == 'StSBC Launch - EVG']; tag_item
tag_id = tag_item[0] if tag_item else None; tag_id

In [ ]:
# Add tag to user

url = f'https://api.convertkit.com/v3/tags/{tag_id}/subscribe'; url
data = json.dumps({'api_key': API_KEY, 'email': 'dnishiyama+test@gmail.com'}); data #parse.quote_plus(
resp = requests.post(url=url, data=data, headers={'content-type': 'application/json'})
if resp.status_code != 200: 
    print(r.status_code) 
    print(r.text)

In [ ]:
# Create Subscription Webhook
# May have 2 and need to delete 997875 (created second)

url = f'https://api.convertkit.com/v3/automations/hooks'; url
target_url = 'https://td9rz0ja7b.execute-api.us-east-1.amazonaws.com/prod/tag_add?tag_id=1028633&tag_name=StSBC+Refunded'; target_url
event = { "name": "subscriber.tag_add", "tag_id": 1028633 }; event
data = json.dumps({'api_secret': CK_API_SECRET, 'target_url': target_url, 'event': event}); data 
resp = requests.post(url=url, data=data, headers={'content-type': 'application/json'})
print(resp.status_code) 
print(resp.text)

In [ ]:
# Create tag-add Webhook
url = f'https://api.convertkit.com/v3/automations/hooks'; url
target_url = 'https://td9rz0ja7b.execute-api.us-east-1.amazonaws.com/prod/tag_add?tag_id=291275&tag_name=Purchased:+Sketchbook+to+Style'; target_url
event = {'name' : 'subscriber.tag_add', 'tag_id': 291275}
data = json.dumps({'api_secret': CK_API_SECRET, 'target_url': target_url, 'event': event}); data 
resp = requests.post(url=url, data=data, headers={'content-type': 'application/json'})
print(resp.status_code) 
print(resp.text)

In [12]:
# Refresher for "Could not confirm type of CK_Api.ALL_SUBS, CK_Api.SEQ_SUBS, False"
from modules.ck_utils import *

In [ ]:
# Some testing
# cursor.execute('SELECT * FROM launch_data WHERE id = 435385229'); cursor.fetchall()
# cursor.quick_info('launch_data')

In [ ]:
# Create unsubscription Webhook (id: 997911)

url = f'https://api.convertkit.com/v3/automations/hooks'; url
target_url = 'https://td9rz0ja7b.execute-api.us-east-1.amazonaws.com/prod/unsubscription'; target_url
event = {'name' : 'subscriber.subscriber_unsubscribe'}
data = json.dumps({'api_secret': CK_API_SECRET, 'target_url': target_url, 'event': event}); data 
resp = requests.post(url=url, data=data, headers={'content-type': 'application/json'})
print(resp.status_code) 
print(resp.text)

In [ ]:
aws_sub_data = {'subscriber': {'id': '435385229_2', 'first_name': 'Declan', 'email_address': 'dnishiyama+test@gmail.com', 'state': 'active', 'created_at': '2019-04-08T19:12:24.000Z', 'fields': {'fb_audience_source': 'Lookalike (WW, 2%) - MC List 4-18-19', 'last_name': '', 'lead_opt-in': 'Influence Map', 'lead_source': 'FB Ad', 'loc_city': 'West Jefferson', 'loc_lat': '36.404', 'loc_lng': '-81.493', 'loc_state': 'North Carolina', 'mailchimp_sub_date': '', 'stsbc_evg_launch_deadline': '11:59pm EDT on Friday, May 31, 2019', 'subscribe_date': '2019-06-04', 'unsubscribe_date': ''}}}

## Location

### Update CK subscriber fields with locations via scraping

In [ ]:
# High-level function
update_subs_with_scraped_locations(smart_stop=True) #smart_stop=False to get ALL locations

### Update lat & lng fields in df

In [ ]:
# Loading Existing data
file_name = 'ck_data'
df = pd.read_pickle(file_name)
print('Length of the indices', len(list(set(df.index.tolist())))) # 50 during error

In [ ]:
# Get the last updated subscriber in the old Dataframe
last_updated_sub = df[df['city'].notnull() | df['state'].notnull()].iloc[::-1].iloc[0]['index']; last_updated_sub

In [ ]:
# Get the new subscriber data from CK API
subs = get_sub_data_for_locless(last_sub=last_updated_sub)

In [ ]:
# Make a Dataframe of that information
append_df = pd.DataFrame([[d[1],d[2]] for d in subs], columns=['city','state'], index=[d[0] for d in subs]); append_df
append_df['lat'] = None; append_df['lng'] = None
append_df.loc[append_df['city'] == 'null', 'city'] = None
append_df.loc[append_df['state'] == 'null', 'state'] = None
append_df = append_df.reset_index()

In [ ]:
# Combine the dfs, make new index, drop duplicated
print(len(df), '+', len(append_df), '=')
df = pd.concat([df, append_df]).drop_duplicates(subset=['index'], keep='first').reset_index(drop=True)
print(len(df))

In [ ]:
# Set the longitude and latitude
geolocator = Nominatim(user_agent="my_app")
for i, row in enumerate(df.iterrows()):
    if row[1].lat or row[1].lng: continue
    print (i, end='\r')
    
    if row[1].city or row[1].state: # Only look it up if they are both not blank
        try:
            coord = get_lat_and_lng(loc_city=row[1].city, loc_state=row[1].state, geolocator=geolocator)
            df.loc[row[0],'lat'] = coord['lat']; df.loc[row[0],'lng'] = coord['lng']
        except LookupError as l:
            print(l)

In [ ]:
# Save the file
file_name = 'ck_data'
df.to_pickle(file_name)

In [ ]:
test = geolocator.reverse("52.509669, 13.376294"); test

In [ ]:
test.raw['address']['country_code']

### Misc Location Updates

In [ ]:
iso3166.countries.get('uk')
# end = re.compile(r'.*?\.([^\.]+$)')
# end.match('@go.cambridgecollege.edu').group(1)

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE id > 20000 AND loc_state="null" and loc_city="null"'); len(cursor.fetchall())

In [ ]:
loc_updates = {}
# See if email addresses of people with no location can help us determine their location
ea = re.compile(r'[^\@]+(\@.+)')
end = re.compile(r'.*?\.([^\.]+$)')
cursor.execute('SELECT * FROM subscriber_data WHERE id > 20000 AND (loc_state IS NULL OR loc_state="null") and (loc_city IS NULL OR loc_city="null")')
for subscriber in cursor.fetchall():
    if not subscriber['email_address']: continue
        
    country_code = end.match(subscriber['email_address']).group(1)
    if len(country_code) != 2: 
        continue
    elif country_code == 'uk': 
        country_name = 'United Kingdom'
    elif country_code == 'pu': 
        country_name = 'India'
    elif country_code in ['eu', 'xo']: 
        continue
    else:
        country_name = iso3166.countries.get(country_code).name
    
    loc_updates[subscriber['id']] = country_name    

In [ ]:
cursor.executemany('UPDATE subscriber_data SET loc_state=%s WHERE id=%s', [[v,k] for k,v in loc_updates.items()])

In [ ]:
# Store new RDS locations in convertkit
session = requests.session()
for k,v in loc_updates.items():
    update_sub_with_misc(k, session=session, loc_state=v)
    

Now check on the lat and lng

In [ ]:
sql_stmt = """
SELECT * FROM subscriber_data 
WHERE (loc_lat IS NULL OR loc_lng IS NULL)
AND ((loc_city IS NOT NULL AND loc_city != 'null') OR (loc_state IS NOT NULL AND loc_state != 'null'))
"""
cursor.execute(sql_stmt); data = cursor.fetchall(); len(data)

In [ ]:
geolocator = Nominatim(user_agent="ck_dgn_locator")
for i, sub in enumerate(data[380:]):
    print(i, end='\r')
    latlng = None
    while not latlng:
        try:
            latlng = get_lat_and_lng(loc_city=sub['loc_city'], loc_state=sub['loc_state'], geolocator=geolocator)
        except Exception as e:
            print(e, "sleeping on #", i)
            time.sleep(10)
    sub['loc_lat'] = latlng['lat']
    sub['loc_lng'] = latlng['lng']

In [ ]:
def test(**kwargs):
    print(kwargs)
sub_temp = {'loc_lat': 'a', 'loc_lng': 'a'}
test(**sub_temp)

In [ ]:
sub_temp

In [ ]:
session = requests.session()
for sub in data:
    if sub['id'] < 20000: continue
    sub_temp = {'loc_lat': str(sub['loc_lat']), 'loc_lng': str(sub['loc_lng'])}
    print(update_sub_with_misc(sub['id'], session=session, **sub_temp))

In [ ]:
sql_stmt = """
UPDATE subscriber_data 
SET loc_lat=%s, loc_lng=%s
WHERE id=%s
"""
cursor.executemany(sql_stmt, [[d['loc_lat'], d['loc_lng'], d['id']] for d in data])

### Add country to MYSQL

In [ ]:
d = lambda pur: datetime.strptime(pur, '%Y-%m-%d') # Date conversion

subs = cursor.e("SELECT * FROM subscriber_data") # Get all subscribers
purchases = cursor.e("SELECT * FROM purchase_data") # Get all purchases
purchases = [{'purchase_date': d(p['purchase_date']) if p['purchase_date'] else None, **p} for p in purchases]
launches = cursor.e("SELECT * FROM launch_data")
locs = [[s['id'], (float(s['loc_lat']), float(s['loc_lng']))] for s in subs if s['loc_lat'] and s['loc_lng']]
ids, coords = zip(*locs)
countries = [iso3166.countries_by_alpha2.get(r['cc'], None).name for r in rg.search(coords, mode=1)]; len(countries)
country_dict = {a[0]:a[1] for a in zip(ids,countries)} #dictionary of countries for subscriber id
# subs = [{**s, 'country': country_dict.get(s['id'], None)} for s in subs]
# sub_dict = {s['id']:s for s in subs}; len(countries) # Dictionary by id

In [ ]:
country_dict.get(list(country_dict)[-1], None)

In [ ]:
[[v,k] for k,v in country_dict.items()][:3]

In [ ]:
values = [[v,k] for k,v in country_dict.items()]
step = 100
start = 0
for i in range(start, len(values)-1, step):
    updated = cursor.e('UPDATE subscriber_data SET loc_country=%s WHERE id=%s', values=values[i:i+step], many=True)
    print(f'Start:{i}, end:{i+step}; updated {updated} rows', end='\r')
    

### Update Countries in Mysql and CK

#### Mysql

In [ ]:
subs = cursor.e('SELECT * FROM subscriber_data WHERE loc_country IS NULL AND ((loc_city IS NOT NULL AND loc_city != "null") OR (loc_state != "null" AND loc_state IS NOT NULL))')
len(subs)
locs = [[s['id'], (float(s['loc_lat']), float(s['loc_lng']))] for s in subs if s['loc_lat'] and s['loc_lng']]
ids, coords = zip(*locs)
countries = [iso3166.countries_by_alpha2.get(r['cc'], None).name for r in rg.search(coords, mode=1)]; len(countries)
country_dict = {a[0]:a[1] for a in zip(ids,countries)} #dictionary of countries for subscriber id
values = [[v,k] for k,v in country_dict.items()]; len(values), values[0]

In [ ]:
step = 100; start = 0
for i in range(start, len(values)-1, step):
    updated = cursor.e('UPDATE subscriber_data SET loc_country=%s WHERE id=%s', values=values[i:i+step], many=True)
    print(f'Start:{i}, end:{i+step}; updated {updated} rows', end='\r')

In [ ]:
conn.commit()

#### CK

In [ ]:
# RDS Data
# conn.commit()
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

log_level('w')

# Takes about 2 minutes
all_ck_list = get_api_data(CK_Api.ALL_SUBS, None, session=session)
all_ck_dict = {a['id']: a for a in all_ck_list}

print(all_rds_list[0]); print(); print(all_ck_list[0])

In [ ]:
all(key=='loc_country' for key in array_mismatch(all_rds_dict[137632787], all_ck_dict[137632787]).keys())

In [ ]:
# Diagnose
a = country_mismatches[0]
array_mismatch(all_rds_dict[a], all_ck_dict[a])
# all(key=='loc_country' for key in array_mismatch(all_rds_dict[a], all_ck_dict[a]).keys())

In [ ]:
country_mismatches = [a for a in all_rds_dict if a in all_ck_dict and all_rds_dict[a]['loc_country'] and 'loc_country' in array_mismatch(all_rds_dict[a], all_ck_dict[a])]
len(country_mismatches)

In [ ]:
try:
    for i, a in enumerate(country_mismatches):
        
        rds_country = all_rds_dict[a]['loc_country']
        ck_country = all_ck_dict[a]['loc_country']
        print(f'#{i}, updating {a}\n{ck_country} to {rds_country}', end='\t\t\t\r')
        
        fields = {'loc_country': rds_country}
        update_ck_sub_with_misc(a, session=session, **fields)
except KeyboardInterrupt as k:
    pass # Allow keyboard break

## Load data into AWS Database

### Create table

In [ ]:
cursor.execute("""
    CREATE TABLE `TEST_subscriber_data` (
      `id` int UNSIGNED NOT NULL,
      `state` varchar(11) NOT NULL,
      `first_name` text,
      `created_at` text,
      `fb_audience_source` text,
      `loc_city` text,
      `loc_state` text,
      `loc_lat` decimal(6,3),
      `loc_lng` decimal(6,3),
      `lead_opt-in` text,
      `lead_source` text,
      `mailchimp_sub_date` text,
      `stsbc_evg_launch_deadline` text,
      `unsubscribe_date` text,
      PRIMARY KEY (`id`));"""
    )

### Dataframe creation and manipulation

In [ ]:
customer_data = get_all_sub_data(columns=[CK_Column.ID, CK_Column.EMAIL_ADDRESS]); customer_data

In [ ]:
new_df = pd.DataFrame([[*d[1:]] for d in customer_data], columns=[c.value for c in columns[1:]], index=[d[0] for d in customer_data]);
new_df['lat'] = None; new_df['lng'] = None; new_df

In [ ]:
new_df.update(df.drop(columns=['loc_city', 'loc_state']))

In [ ]:
new_df.to_pickle('temp')

In [ ]:
get_single_subscriber(137632665)

MYSQL Commands

In [ ]:
# TEST MYSQL COMMANDS
# cursor.execute('SELECT * FROM subscriber_data WHERE loc_state LIKE "%?%"'); cursor.fetchall() 
cursor.execute('SELECT * FROM subscriber_data WHERE id=471301153'); cursor.fetchall()
# cursor.execute('SELECT * FROM subscriber_data WHERE loc_city="null"'); cursor.fetchall()
# cursor.execute('DELETE FROM subscriber_data WHERE id=137632665')
# cursor.execute('SELECT COUNT(*) as count FROM subscriber_data'); cursor.fetchall()
# cursor.execute('SELECT default_character_set_name FROM information_schema.SCHEMATA S WHERE schema_name = "ck_info";'); cursor.fetchall()
# cursor.execute('SELECT CCSA.character_set_name FROM information_schema.`TABLES` T,information_schema.`COLLATION_CHARACTER_SET_APPLICABILITY` CCSA WHERE CCSA.collation_name = T.table_collation AND T.table_schema = "ck_info" AND T.table_name = "TEST_subscriber_data";'); cursor.fetchall()
# cursor.execute('ALTER TABLE subscriber_data CONVERT TO CHARACTER SET utf8;')
# cursor.execute('ALTER DATABASE ck_info CHARACTER SET utf8 COLLATE utf8_general_ci;')
# conn.commit()

In [ ]:
# Problem with "?" instead of utf-8, # Example 467114222

In [ ]:
# INSERT ON DUPLICATE KEY UPDATE testing
values = [137632665,None,'active','2017-10-01T08:08:17.000Z',None,'ספר','ספר','Kuala Lumpur','Kuala Lumpur',
 '2017-10-01 04:08:51',None, None]; values
cursor.execute("""
INSERT into TEST_subscriber_data 
(`id`, `first_name`, `state`, `created_at`, `fb_audience_source`, `lead_opt-in`, `lead_source`, `loc_city`, `loc_state`, `mailchimp_sub_date`, `stsbc_evg_launch_deadline`, `unsubscribe_date`)
values 
(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)
ON DUPLICATE KEY UPDATE 
id  = VALUES(id), 
fb_audience_source  = VALUES(fb_audience_source), 
`lead_opt-in`  = VALUES(`lead_opt-in`), 
lead_source  = VALUES(lead_source), 
stsbc_evg_launch_deadline = VALUES(stsbc_evg_launch_deadline),
loc_city  = VALUES(loc_city), 
loc_state   = VALUES(loc_state),
unsubscribe_date = VALUES(unsubscribe_date);
""", values);

In [ ]:
# Update RDS with most recent subscriber information
non_mysql = ["email_address", "last_name"]

    # Get all subscribers from the CK API 
subs = get_all_sub_data(columns=[], dictionary=True, test=False); subs

    # Prep columns and values
columns, values = zip(*list(subs[0].items())); columns
column_string = ", ".join(["`"+col+"`" for col in columns if col not in non_mysql]); column_string
variable_string = ", ".join(["%s"]*12); variable_string
duplicate_string = "ON DUPLICATE KEY UPDATE "+", ".join(["`"+col+"`=VALUES(`"+col+"`)" for col in columns if col not in non_mysql]); duplicate_string

    # Prep statement
sql_string = f'insert ignore into subscriber_data ({column_string}) values ({variable_string}) {duplicate_string};'; sql_string
values = [[v for k,v in sub.items() if k not in non_mysql] for sub in subs]; values

    # Add those subscribers to the database
cursor.executemany(sql_string, values)
    

In [ ]:
conn.commit()

In [ ]:
cursor.execute('UPDATE subscriber_data SET loc_lat=1, loc_lng=1, loc_state="Ashland", loc_city="Kentucky" WHERE id=467247086');


In [ ]:
[d for d in data if not d['loc_city'] and not d['loc_state']][0]

In [ ]:
get_lat_and_lng(loc_city='Ashland', loc_state='kentucky', geolocator = Nominatim(user_agent="dgn_ck_geolocator"))

In [ ]:
str_but_no_latlng[35:37]

In [ ]:
ac_conn = pymysql.connect(
    user=os.environ['RDS_CK_USER'], 
    password=os.environ['RDS_CK_PASSWORD'], 
    host=os.environ['RDS_CK_HOST'],
    database='ck_info',
#     charset='utf8',
    cursorclass=pymysql.cursors.DictCursor,
    autocommit=True
)
ac_cursor = ac_conn.cursor()

In [ ]:
ac_cursor.execute('SELECT * FROM subscriber_data WHERE loc_state!="null"'); data = ac_cursor.fetchall()
str_but_no_latlng = [d for d in data if ((d['loc_city'] and d['loc_city']!='null') or (d['loc_state'] and d['loc_state']!='null')) and not d['loc_lat']]
print(len(data), len(str_but_no_latlng))


In [ ]:
geolocator = Nominatim(user_agent="dgn_ck_geolocator")
skip_to=0
for i, sub in enumerate(str_but_no_latlng[skip_to:]):
    lc = sub['loc_city']; ls = sub['loc_state']; lid = sub['id']
    time.sleep(1)
    try:
        latlng = get_lat_and_lng(loc_city=lc, loc_state=ls, geolocator=geolocator)
    except LookupError as e:
        print(f'LookupError for {lid} - {sub["loc_city"]}, {sub["loc_state"]}')
        continue
    except Exception as e:
        print(i+skip_to, 'TIMEOUT')
        time.sleep(20)
        latlng = get_lat_and_lng(loc_city=lc, loc_state=ls, geolocator=geolocator)
    lat = latlng['lat']; lng = latlng['lng']
    ac_cursor.execute(f'UPDATE subscriber_data SET loc_lat={lat}, loc_lng={lng} WHERE id={lid}')
    print('-'*100, end='\r')
    print(i+skip_to, lid, lc, ls, latlng, end='\r')

In [ ]:
ac_cursor.execute('SELECT * FROM subscriber_data WHERE id=463137395'); ac_cursor.fetchall()

In [ ]:
ac_cursor.execute('SELECT * FROM subscriber_data'); subscribers = ac_cursor.fetchall()

In [ ]:
[[i,s] for i, s in enumerate(subscribers) if s['id']==137632833]

In [ ]:
subscribers[100]['loc_lat']

In [ ]:
# Load RDS subscriber lat and lng back 
for i, sub in enumerate(subscribers):
    sub_id = sub['id']
    lat = str(sub['loc_lat']) if sub['loc_lat'] else 'null'
    lng = str(sub['loc_lng']) if sub['loc_lng'] else 'null'
    print(f'{i}/{len(subscribers)}', end='\r')
    
    
#     if r.status_code 

Load RDS subscriber lat and lng back async

In [ ]:
session = requests.session() # Create session
cursor.execute('SELECT id, loc_lat, loc_lng, subscribe_date FROM subscriber_data'); subscribers = cursor.fetchall()
sub_ids = [sub['id'] for sub in subscribers]
lats = [str(sub['loc_lat']) if sub['loc_lat'] else 'null' for sub in subscribers]
lngs = [str(sub['loc_lng']) if sub['loc_lng'] else 'null' for sub in subscribers]
sub_date = [sub['subscribe_date'] for sub in subscribers]
subscribers = [s for s in list(zip(sub_ids, lats, lngs, sub_date)) if s[0] > 20000]; len(subscribers), subscribers[:3]

In [ ]:
async def work(data_array):
    sub_id, lat, lng, sub_date = data_array
    url = f'https://api.convertkit.com/v3/subscribers/{sub_id}'
    data = json.dumps({'api_secret': CK_API_SECRET,'fields':{'loc_lat': lat, 'loc_lng': lng, 'subscribe_date': sub_date}})
    headers = {'content-type': 'application/json'}
    r = session.put(url=url, data=data, headers=headers)
    if r.status_code != 200: 
        print('ERROR!', sub)
        
await asyncio.gather(*[work(sub) for sub in subscribers[40:]]);

In [ ]:
subscribers.index(sub)

In [ ]:
notify('done!')

### Inserting ck export into RDS

In [ ]:
# Get the csv and use it
ck_customer_data = prep_ck_csv("/Users/nish/Downloads/2019-05-21-9752.csv"); ck_customer_data.loc[0];

In [ ]:
# ck_customer_data.to_csv("/Users/nish/Downloads/all_convertkit_subscriber_data.csv")

In [ ]:
# Add "id" column and remove "nan" processing
ck_customer_data['id'] = ck_customer_data['email'].map(subscribers)
ck_customer_data = ck_customer_data.where((pd.notnull(ck_customer_data)), None)

In [ ]:
# Set empty ids to an int
i = 0
for row in ck_customer_data.iterrows():
#     print(row[1])
#     ck_customer_data[]
    if not ck_customer_data.loc[row[0], 'id']: 
        ck_customer_data.loc[row[0], 'id'] = i
        i += 1

In [ ]:
    # Prep columns and values
columns = ck_customer_data.columns
values = ck_customer_data.values.tolist(); columns, values
column_string = ", ".join(["`"+col+"`" for col in columns]); column_string
variable_string = ", ".join(["%s"]*len(columns)); variable_string
duplicate_string = "ON DUPLICATE KEY UPDATE `email`=VALUES(`email`)"; duplicate_string #update for existing

    # Prep statement
sql_string = f'insert into subscriber_data ({column_string}) values ({variable_string}) {duplicate_string};'; sql_string

    # Add those subscribers to the database
cursor.executemany(sql_string, values)
    

In [ ]:
cursor.execute('INSERT IGNORE INTO TEST_subscriber_data (first_name, id) VALUES (%s, %s)', [float('nan'), 2])
cursor.execute('SELECT * FROM TEST_subscriber_data'); cursor.fetchall()

In [ ]:
get_single_subscriber(email_address = 'dnishiyama@gmail.com')

In [ ]:
[[v[0], ] for v in ck_customer_data.loc[:3].values]

In [ ]:
cursor.execute('Select * FROM subscriber_data WHERE email LIKE "%@%"'); len(cursor.fetchall())

In [ ]:
# Get the different types of states
# {'active', 'bounced', 'cancelled', 'complained', 'inactive'}
actives = ck_customer_data['status']=='active'
bounceds = ck_customer_data['status']=='bounced'
cancelleds = ck_customer_data['status']=='cancelled'
complaineds = ck_customer_data['status']=='complained'
inactive = ck_customer_data['status']=='inactive'
subs_to_fetch = ck_customer_data[actives | bounceds | cancelleds]['email'].tolist()

In [ ]:
subs_to_fetch = ck_customer_data['email'].tolist(); len(subs_to_fetch)

In [ ]:
# Get all ids from Convertkit sequentially
session=requests.session()
start = 0
for i,email in enumerate(ck_customer_data['email'].tolist()[start:]):
    if not email: continue
    sub_id=None
    subs = get_single_subscriber(session=session, email_address=email).get('subscribers', [])
    if subs and 'id' in subs[0]: sub_id = subs[0]['id']
    subscribers.update({email: sub_id})
    print(i+start, end='\r')

In [ ]:
# Missing emails
[c for c in ck_customer_data['email'].tolist() if c not in subscribers]

In [ ]:
# Number of missing ids (inactives etc.)
len([c for c in ck_customer_data['email'].tolist() if c and not subscribers[c]])

In [ ]:
# Get data from convertkit API asynchronously
session = requests.session()

async def work(email_address):
    sub_id = None
    subs = get_single_subscriber(session=session, email_address=email_address).get('subscribers', [])
    if subs and 'id' in subs[0]: sub_id = subs[0]['id']
    return {email_address: sub_id}

data = await asyncio.gather(*[work(sub) for sub in subs_to_fetch])

subscribers.update({k:v for dicts in data for k,v in dicts.items()})

### Add MailChimp subscribe dates

In [ ]:
# Alter table
# cursor.execute('ALTER TABLE subscriber_data ADD COLUMN subscribe_date text AFTER stsbc_evg_launch_deadline')
# cursor.execute('UPDATE subscriber_data SET `subscribe_date`=`created_at`')

In [ ]:
# See the existing data type
# cursor.execute('SELECT created_at FROM subscriber_data LIMIT 1'); cursor.fetchall()
# cursor.execute('INSERT INTO TEST_subscriber_data (`id`, `email`, `state`) VALUES (3, "sguinnie@hotmail.com", "active")'); cursor.fetchall()
# cursor.execute('SELECT * FROM subscriber_data WHERE id = 474237317'); cursor.fetchall()
# cursor.execute('SELECT * FROM subscriber_data WHERE mailchimp_sub_date IS NOT NULL LIMIT 1'); cursor.fetchall()


In [ ]:
# Get the csv and make a 2 column dataframe from it
ck_customer_data = prep_ck_csv("/Users/nish/Downloads/sheets_ck_data.csv"); ck_customer_data
ck_customer_data = ck_customer_data[['email', 'mailchimp_sub_date']]; ck_customer_data
ck_customer_data = ck_customer_data[ck_customer_data['mailchimp_sub_date'].notnull()]; ck_customer_data
ck_customer_data['mailchimp_sub_date'] = ck_customer_data['mailchimp_sub_date'].apply(convert_time)

In [ ]:
# Make a new column based on the data in the table with reformatting
cursor.execute('SELECT created_at, id FROM subscriber_data')
created_at_dates = cursor.fetchall(); created_at_dates
conv = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z').strftime('%Y-%m-%d')
created_at_dates = [[conv(c['created_at']), c['id']] for c in created_at_dates]; created_at_dates[:3]

In [ ]:
# cursor.executemany('UPDATE subscriber_data SET subscribe_date=%s WHERE id=%s', created_at_dates[18000:])

In [ ]:
conn.commit()

### Add MailChimp subscribers

In [ ]:
# Get the data from RDS
cursor.execute('SELECT * FROM subscriber_data'); rds_subscribers = cursor.fetchall()
print([c[0] for c in cursor.description])
cursor.execute('SELECT * FROM purchase_data'); rds_purchasers = cursor.fetchall()
rds_sub_emails = [r['email_address'] for r in rds_subscribers]; print(len(rds_sub_emails))
rds_pur_emails = [r['email_address'] for r in rds_purchasers]; print(len(rds_pur_emails))

In [ ]:
pd.options.mode.chained_assignment = None

# Get and prep the csv from MailChimp (sub and unsub combined)
mc_df = pd.read_csv('/Users/nish/Downloads/mailchimp_subscribers.csv'); mc_df
mc_df = mc_df[['Email Address', 'First Name', 'LATITUDE', 'LONGITUDE', 'OPTIN_TIME', 'UNSUB_TIME']] # reduce cols
mc_df.columns = ['email_address', 'first_name', 'loc_lat', 'loc_lng', 'subscribe_date', 'unsubscribe_date']
mc_df = mc_df.where((pd.notnull(mc_df)), None); mc_df # change nan to Nones
mc_df['email_address'] = [e.lower() for e in mc_df['email_address'].tolist()]; mc_df.head(3) # make emails lowercase
datetime_conv = lambda x: datetime.strptime(x, '%m/%d/%y %H:%M').strftime('%Y-%m-%d')
mc_df['subscribe_date'] = [datetime_conv(e) if e else None for e in mc_df['subscribe_date'].tolist()]
mc_df['unsubscribe_date'] = [datetime_conv(e) if e else '2019-04-08' for e in mc_df['unsubscribe_date'].tolist()]
mc_df['loc_lat'] = [round(e, 3) if e else None for e in mc_df['loc_lat'].tolist()]
mc_df['loc_lng'] = [round(e, 3) if e else None for e in mc_df['loc_lng'].tolist()]
mc_df['state'] = 'cancelled'; mc_df.head(2)
mc_df = mc_df.where((pd.notnull(mc_df)), None); mc_df.head(2) # change nan to Nones
mc_df_2 = mc_df[~mc_df['email_address'].isin(rds_sub_emails)]; print(len(mc_df_2)); mc_df_2.head(2)

In [ ]:
# Convert the lng and lat to places
def get_place(coord):
    if not coord[0] or not coord[1]: return coord
    place = rg.search(coord, mode=1)
    city = place[0]['admin1']
    country = iso3166.countries_by_alpha2.get(place[0]['cc']).name
    return city, country
    
coords = list(zip(mc_df_2['loc_lat'], mc_df_2['loc_lng'])); coords
places = [get_place(c) if c else None for c in coords]; places
city_col, state_col = list(zip(*[p for p in places])); city_col[:3], state_col[:3]

In [ ]:
# Add the city and state back to the table

mc_df_2['loc_city']=pd.Series(city_col).values
mc_df_2['loc_state']=pd.Series(state_col).values
mc_df_2.head(2)

In [ ]:
cursor.execute('SELECT max(id) as m FROM subscriber_data WHERE id < 20000')
max_id = cursor.fetchall()[0]['m']; max_id

In [ ]:
cols = ['id'] + mc_df_2.columns.tolist()
values = [[max_id+1+i, *list(v)] for i, v in enumerate(mc_df_2.to_numpy())]; values
sql_stmt = f"""\
INSERT INTO subscriber_data
({", ".join(cols)})
VALUES
({", ".join(["%s"]*len(cols))})
"""; sql_stmt
cursor.executemany(sql_stmt, values)

In [ ]:
conn.commit()

In [ ]:
### Add the new mc emails to the subscribers table
# Get the emails, and other data for people that don't exist in subscribers
# Add them to subscribers
# Check to see if the purchases need to be updated

In [ ]:
[r for r in rds_purchasers if r['subscriber_id'] < 20000 and r['email_address'] in mc_df_2['email_address'].tolist()]

### Check CK vs RDS

Missing items from RDS

In [ ]:
ck_sub_data = get_all_sub_data(columns=[]); ck_sub_data[-1]

In [ ]:
column_string = ', '.join(['`'+c.value+'`' for c in CK_Column]); column_string
cursor.execute(f'SELECT {column_string} FROM subscriber_data'); 
rds_sub_data = cursor.fetchall(); len(rds_sub_data)

In [ ]:
rds_ids = [c['id'] for c in rds_sub_data]; len(rds_ids), rds_ids[-3:]
ck_ids = [c[0] for c in ck_sub_data]; len(ck_ids), ck_ids[-3:]

In [ ]:
result = list(set(ck_ids) - set(rds_ids)); result

In [ ]:
update_single_subscriber(sub_id=474316521, conn=conn)

Missing data

In [ ]:
rds_sub_data[0]

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE mailchimp_sub_date IS NOT NULL AND unsubscribe_date IS NULL')
data = cursor.fetchall(); len(data)#, data[0]

In [ ]:
conv_created_at_to_subscribe_date(data[-1]['created_at'])

In [ ]:
values = [[conv_created_at_to_subscribe_date(d['created_at']), d['id']] for d in data]; values

In [ ]:
cursor.executemany('UPDATE subscriber_data SET subscribe_date=%s WHERE id=%s', values)

In [ ]:
conn.commit()

### Purchases Database


In [ ]:
cursor.execute('SHOW TABLES;'); cursor.fetchall()

### Fix lack of emails 

In [ ]:
# sql = 'SELECT id, email_address, loc_lat, loc_lng, STR_TO_DATE(created_at, "%Y-%m-%dT%H:%i:%s") as d FROM subscriber_data HAVING d >= NOW() - INTERVAL 1 DAY'
sql = 'SELECT * FROM subscriber_data WHERE email_address IS NULL AND id > 20000'
cursor.execute(sql); data = cursor.fetchall(); len(data)

In [ ]:
session = requests.session()
fixes = {}
for i, d in enumerate(data):
    print(i, end='\r')
    sub_data = get_single_subscriber(session, subscriber_id=d['id'])
    fixes[d['id']] = sub_data['email_address']

In [ ]:
values = [[v,k] for k,v in fixes.items()]; values[:3]
cursor.executemany('UPDATE subscriber_data SET email_address=%s WHERE id=%s', values)
conn.commit()

### Create Link Shortener

#### Mysql

In [ ]:
cursor.execute("""
    CREATE TABLE `links` (
      `id` MEDIUMINT NOT NULL AUTO_INCREMENT,
      `char_value` CHAR(1) NOT NULL,
      `char_num` SMALLINT UNSIGNED NOT NULL,
      `link_value_position` SMALLINT UNSIGNED NOT NULL,
      `link_value` text,
      `description` text,
      PRIMARY KEY (`id`));"""
    )

In [ ]:
cursor.execute("""
    CREATE TABLE `link_shortener` (
      `id` CHAR(1) NOT NULL,
      `char_num` INT UNSIGNED NOT NULL,
      `value_position` INT UNSIGNED NOT NULL,
      `value` text,
      `description` text,
      PRIMARY KEY (`id`, `char_num`, `value_position`));"""
    )

In [ ]:
cursor.execute('drop table link_shortener')

In [ ]:
cursor.execute('show tables;'); cursor.fetchall()

In [ ]:
a = """
            case 'a', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break; // WAF Activity Guide
            case 'b', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break; // Book Making Guide
            case 'e', 0, 0: longUrl += 'might-could.com'; break; // Emails (NO LANDING PAGE EXISTS)
            case 'f', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break; //WAF ebook
            case 'i', 0, 0: longUrl += 'lp.mightcouldstudios.com'; break; // Influence Map
            case 'k', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break // Influence Map Sketchbook
            case 'm', 0, 0: longUrl += 'lp.mightcouldstudios.com'; break; // MCDT
            case 's', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break; // StSBC WL
            case 't', 0, 0: longUrl += 'mightcouldstudios.lpages.co'; break; // Tools
            case 'u', 0, 0: longUrl += 'might-could.com'; break; // Studiomates WL (NO LANDING PAGE EXISTS)
            case 'y', 0, 0: longUrl += 'lp.mightcouldstudios.com'; break; // Style Mini-course
            case 'a', 0, 1: longUrl += 'we-are-fungi-ebook-activity-guide/'; break; // WAF Activity Guide
            case 'b', 0, 1: longUrl += 'storyboarding-your-book-guide/'; break; // Book Making Guide
            case 'e', 0, 1: longUrl += 'emails/'; break; // Emails (NO LANDING PAGE EXISTS)
            case 'f', 0, 1: longUrl += 'we-are-fungi-ebook/'; break; //WAF ebook
            case 'i', 0, 1: longUrl += 'influence-map/'; break; // Influence Map
            case 'k', 0, 1: longUrl += 'how-to-draw-more-than-ever-before-2/'; break // Influence Map Sketchbook
            case 'm', 0, 1: longUrl += 'mightcoulddrawtoday/'; break; // MCDT
            case 's', 0, 1: longUrl += 'sketchbook-to-style-boot-camp-waitlist/'; break; // StSBC WL
            case 't', 0, 1: longUrl += 'tools-i-use-and-love/'; break; // Tools
            case 'u', 0, 1: longUrl += 'studiomates-wl/'; break; // Studiomates WL (NO LANDING PAGE EXISTS)
            case 'y', 0, 1: longUrl += 'style-mini-course/'; break; // Style Mini-course
            case 'a', 1, 2: longUrl += 'FB%20Ad'; break; // FB Ad
            case 'f', 1, 2: longUrl += 'Facebook'; break; // Facebook
            case 'i', 1, 2: longUrl += 'Instgram'; break; // Instagram
            case 'm', 1, 2: longUrl += 'Medium'; break; // Medium
            case 'n', 1, 2: longUrl += 'Manual'; break; // Manual
            case 'p', 1, 2: longUrl += 'MCS%20Page'; break; // MCS Page
            case 'r', 1, 2: longUrl += 'WAF%20Book%20Reading'; break; //WAF Book Reading
            case 's', 1, 2: longUrl += 'Skillshare'; break; // Skillshare
            case 't', 1, 2: longUrl += 'Twitter'; break; // Twitter
"""

In [ ]:
print(a.replace('case ', '').replace(' ', '').replace(':longUrl+=',',').replace(';break;//',',\'').replace('\n','\'],\n ['))

In [ ]:
b = [
 ['a',0,0,'mightcouldstudios.lpages.co','WAF Activity Guide'],
 ['b',0,0,'mightcouldstudios.lpages.co','Book Making Guide'],
 ['e',0,0,'might-could.com','Emails'],
 ['f',0,0,'mightcouldstudios.lpages.co','WAF ebook'],
 ['i',0,0,'lp.mightcouldstudios.com','Influence Map'],
 ['k',0,0,'mightcouldstudios.lpages.co', 'Influence Map Sketchbook'],
 ['m',0,0,'lp.mightcouldstudios.com','MCDT'],
 ['s',0,0,'mightcouldstudios.lpages.co','StSBC WL'],
 ['t',0,0,'mightcouldstudios.lpages.co','Tools'],
 ['u',0,0,'might-could.com','Studiomates WL '],
 ['y',0,0,'lp.mightcouldstudios.com','Style Mini-course'],
 ['a',0,1,'we-are-fungi-ebook-activity-guide','WAF Activity Guide'],
 ['b',0,1,'storyboarding-your-book-guide','Book Making Guide'],
 ['e',0,1,'emails','Emails'],
 ['f',0,1,'we-are-fungi-ebook','WAF ebook'],
 ['i',0,1,'influence-map','Influence Map'],
 ['k',0,1,'how-to-draw-more-than-ever-before-2', 'Influence Map Sketchbook'],
 ['m',0,1,'mightcoulddrawtoday','MCDT'],
 ['s',0,1,'sketchbook-to-style-boot-camp-waitlist','StSBC WL'],
 ['t',0,1,'tools-i-use-and-love','Tools'],
 ['u',0,1,'studiomates-wl','Studiomates WL'],
 ['y',0,1,'style-mini-course','Style Mini-course'],
 ['a',1,2,'FB%20Ad','FB Ad'],
 ['f',1,2,'Facebook','Facebook'],
 ['i',1,2,'Instgram','Instagram'],
 ['m',1,2,'Medium','Medium'],
 ['n',1,2,'Manual','Manual'],
 ['p',1,2,'MCS%20Page','MCS Page'],
 ['r',1,2,'WAF%20Book%20Reading','WAF Book Reading'],
 ['s',1,2,'Skillshare','Skillshare'],
 ['t',1,2,'Twitter','Twitter']]

In [ ]:
      `char_value` CHAR(1) NOT NULL,
      `char_num` SMALLINT UNSIGNED NOT NULL,
      `link_value_position` SMALLINT UNSIGNED NOT NULL,
      `link_value` text,
      `description` text,
      PRIMARY KEY (`id`));"""

In [ ]:
cursor.executemany('INSERT INTO links (`char_value`, `char_num`, `link_value_position`, `link_value`, `description`) VALUES (%s, %s, %s, %s, %s)', b)

In [ ]:
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM links'); cursor.fetchall()

In [ ]:
# cursor.quick_info('link_shortener')
cursor.quick_info('links')

In [ ]:
cursor.execute('SELECT * FROM link_shortener')

#### DynamoDB

In [ ]:
import boto3 
from boto3 import dynamodb 
from boto3.session import Session
from boto3.dynamodb.conditions import Key, Attr
dynamodb_session = Session(aws_access_key_id='',
          aws_secret_access_key='',
          region_name='us-east-1')
dynamodb = dynamodb_session.resource('dynamodb')
table=dynamodb.Table('link_extender')


In [ ]:
table.get_item(TableName='link_extender', Key={'id':'0'})

In [ ]:
cursor.execute('SELECT * FROM link_shortener'); mysql_data = cursor.fetchall(); mysql_data
del mysql_data[3]
del mysql_data[0]
mysql_data[0]

In [ ]:
for i, datum in enumerate(mysql_data):
    datum['char_value'] = datum['id']
    datum['link_value'] = datum['value']
    datum['link_value_position'] = int(datum['value_position'])
    datum['id'] = str(i+2)
    del datum['value']; del datum['value_position']
    table.put_item(TableName='link_extender', Item=datum)

In [ ]:
#  ['b',0,0,'mightcouldstudios.lpages.co','Book Making Guide'],
table.put_item(TableName='link_extender', 
               Item={'id': '1', 
                     'char_value':'b',
                     'char_num': 0,
                     'link_value_position': 0,
                     'link_value': 'mightcouldstudios.lpages.co',
                     'description': 'Book Making Guide'
                    }
                    
              )

In [ ]:
table.scan()['Items']

## Create Map

Get the 10k existing subscribers

In [ ]:
sql_stmt = 'SELECT id,loc_lat,loc_lng FROM subscriber_data WHERE loc_lat IS NOT NULL AND loc_lng IS NOT NULL'
cursor.execute(sql_stmt)
data = cursor.fetchall(); len(data), data[0]

In [ ]:
data_string = ['[' + str(d['loc_lat']) + ', ' + str(d['loc_lng']) + ', ' + str(d['id']) + '],\n' for d in data]
data_string[-1] = data_string[-1].replace(',\n', '\n')

In [ ]:
with open('subscriber_locations.js', 'w') as f:
    f.write('//All subscriber locations gathered before 5/24/19 11:46 ET.\n')
    f.write('//There are 11540 points. Method is in ConvertKit Manipulation jupyter notebook\n')
    f.write('var subscriber_locations = [\n')
    f.writelines(data_string)
    f.write('];')

Other

In [ ]:
#SELECT CAST(orders.date_purchased AS DATE) AS DATE_PURCHASED
try:
    cursor.execute('SELECT STR_TO_DATE(created_at, "%Y-%m-%dT%H:%i:%s") AS d FROM subscriber_data HAVING d >= NOW() - INTERVAL 2 HOUR ORDER BY d DESC'); 
    data = cursor.fetchall()
except Exception as e:
    raise e
data

## Purchases

In [ ]:
cursor.quick_info('purchase_data')
# cursor.quick_info('purchase_data', subscriber_id=447243341)
# cursor.e('DESCRIBE purchase_data')

#### Modify the table

In [ ]:
# cursor.execute('ALTER TABLE purchase_data CHANGE purchase_id purchaser_id text;')
# cursor.execute('ALTER TABLE purchase_data CHANGE purchase_email_address purchaser_email_address VARCHAR(100);')
# cursor.execute('ALTER TABLE purchase_data ADD COLUMN purchaser_id TEXT AFTER subscriber_email_address;')
# cursor.execute('UPDATE purchase_data SET subscriber_email_address = purchaser_email_address')

### Misc Additions

In [ ]:
new_pur = [
    [304637290, 'colette.levesque@gmail.com', 'Sketchbook to Style', '2019-05-12', 'Evergreen_2', 297],
    [464254353, 'diamond.quartz97@gmail.com', 'Sketchbook to Style', '2019-05-15', 'Evergreen_2', 297],
    [461785863, 'lmhkanazawa@gmail.com', 'Sketchbook to Style', '2019-05-16', 'Evergreen_2', 297],
    [471651008, 'iris-stam@hotmail.com', 'Sketchbook to Style', '2019-05-22', 'Evergreen_2', 297],
    [477841123, 'erika.norris@gmail.com', 'Sketchbook to Style', '2019-05-26', 'Evergreen_2', 297]
]; new_pur

In [ ]:
# ['subscriber_id', 'email_address', 'item', 'purchase_date', 'launch', 'purchase_value']
sql_stmt = """INSERT INTO purchase_data
(subscriber_id, email_address, item, purchase_date, launch, purchase_value)
VALUES (%s, %s, %s, %s, %s, %s)
"""; print(sql_stmt)

# sql_stmt = 'SELECT * FROM purchase_data'
cursor.executemany(sql_stmt, new_pur); cursor.fetchall()[-10:]

In [ ]:
sql_stmt = 'SELECT * FROM purchase_data'
cursor.execute(sql_stmt); cursor.fetchall()[-10:]

In [ ]:
conn.commit()

### Fix double entries (in subscriber data)

In [ ]:
# Find double entries
cursor.execute('SELECT id, state, email_address FROM subscriber_data GROUP BY email_address HAVING COUNT(*) > 1')
duplicates = cursor.fetchall(); len(duplicates), duplicates[:3]

In [ ]:
# See if any that will be removed have better info than the remaining items
    # Get the pairs
sub_ids_to_fix=[]
for duplicate in duplicates:
    dup_subs = [s for s in subscribers if s['email_address'] == duplicate['email_address']]
    if dup_subs[0]['loc_lat'] is not None and dup_subs[1]['loc_lat'] is None:
        sub_ids_to_fix.append(dup_subs[1]['id'])
sub_ids_to_fix

In [ ]:
# update_single_subscriber(470921734, conn)

In [ ]:
# Remove the other duplicates

In [ ]:
sub_ids_to_delete = [d['id'] for d in duplicates]; len(sub_ids_to_delete), max(sub_ids_to_delete)

In [ ]:
# Check for purchases that need to be updated
[p for p in purchases if p['subscriber_id'] in sub_ids_to_delete]

In [ ]:
cursor.executemany('DELETE FROM subscriber_data WHERE id = %s', sub_ids_to_delete)

In [ ]:
conn.commit()

### Fixes

#### Add subscriber_email_where possible

In [ ]:
# Update other_email_address in CK and then run these commands

In [ ]:
cursor.e('UPDATE purchase_data SET subscriber_id=433564154, subscriber_email_address="meline.co@gmail.com" WHERE purchaser_email_address="emeline.courcelle@gmail.com"')


In [ ]:
update_ck_sub_with_misc(sub_id=536414538, other_email_address="meline.co@gmail.com")

In [ ]:
# purchases = cursor.e('SELECT * FROM purchase_data WHERE launch="Launch_6"')
sql_stmt = 'SELECT purchaser_id, purchaser_email_address FROM purchase_data WHERE subscriber_email_address IS NULL'
purchasers_with_no_sub = [n for n in cursor.e(sql_stmt)]
len(purchasers_with_no_sub), purchasers_with_no_sub

In [ ]:
# Update the RDS if needed with CK Other email addresses
# for n in purchasers_with_no_sub: 
update_subscriber_in_rds(433564154, conn) 

In [ ]:
# Gather the data for the update in purchase data (sub_id, sub_email, pur_email)
sql_stmt = f"""\
SELECT id, other_email_address, email_address
FROM subscriber_data 
WHERE id IN {mysql_array(no_subs)} 
    AND other_email_address IS NOT NULL
"""
updateables = cursor.e(sql_stmt)
len(updateables), updateables[0]

In [ ]:
sql_stmt='UPDATE purchase_data SET subscriber_id=%s, subscriber_email_address=%s WHERE purchaser_email_address=%s'
values = [[v for k,v in s.items()] for s in updateables]
cursor.e(sql_stmt, values=values, many=True)

In [ ]:
# Update RDS with data from CK for "other_email_address"
# update_subscriber_in_rds(531681162, conn)
# get_single_subscriber(email_address="dalefarwalker@gmail.com")

In [ ]:
logging.getLogger().setLevel(logging.INFO)

#### Update purchasers with correct launch

In [ ]:
dt = lambda ds: datetime.strptime(ds, '%Y-%m-%d')
recent = lambda x: x['purchase_date'] and dt(x['purchase_date']) > datetime.now()-timedelta(days=30)

purchases = cursor.e('SELECT * FROM purchase_data')
recent_purchases = [p for p in purchases if recent(p)]; len(recent_purchases), recent_purchases[:2]

In [ ]:
# Trim purchases
recent_purchases = [p for p in recent_purchases if p['purchase_email_address'] not in ['joanne.carlos2@gmail.com', 'a.molihan@comcast.net', 'pbrosebud@ptd.net']]
recent_purchases

In [ ]:
cursor.e('SELECT DISTINCT launch FROM purchase_data') # See list of launches (to match style)

In [ ]:
cursor.executemany('UPDATE purchase_data SET launch="Launch_6" WHERE purchase_email_address=%s', [p['purchase_email_address'] for p in recent_purchases])

In [ ]:
conn.commit()

In [ ]:
len(cursor.e('SELECT * FROM purchase_data WHERE launch="Launch_6"'))

#### Refunds

In [ ]:
# cursor.e('SELECT * FROM purchase_data WHERE refund = 1')

In [ ]:
# cursor.e('UPDATE purchase_data SET refund=1 WHERE purchaser_email_address="rotxro@gmail.com"')

#### Other

In [ ]:
cursor.e('DELETE FROM purchase_data WHERE purchaser_email_address = "andreemorgan9@mail.com"')

In [ ]:
conn.commit()

### Validation

In [ ]:
# cursor.e('SELECT * FROM purchase_data WHERE purchaser_email_address LIKE "dnishiyama%"')
# cursor.e('SELECT * FROM purchase_data WHERE launch = "Evergreen_2"')
# cursor.e('SELECT * FROM purchase_data WHERE launch = "Launch_6"')
cursor.e('SELECT * FROM purchase_data WHERE refund = 1')

In [ ]:
conn.commit()

## Launches

In [ ]:
cursor.e('SELECT * FROM launch_info')

### Create the tables

In [ ]:
# Create the table with info on launches
cursor.execute("""
    CREATE TABLE `launch_info` (
      `launch` varchar(20) NOT NULL,
      `start` text NOT NULL,
      `duration` int,
      PRIMARY KEY (`launch`));"""
    )

In [ ]:
# Create the table with the data on launches (subscribers)
cursor.execute("""
    CREATE TABLE `launch_data` (
      `id` int UNSIGNED NOT NULL,
      `launch` varchar(20) NOT NULL,
      PRIMARY KEY (`id`, `launch`));"""
    )

In [ ]:
cursor.execute('SELECT * FROM launch_info'); cursor.fetchall()#; cursor.description

In [ ]:
cursor.execute('SHOW TABLES;'); cursor.fetchall()

### Add info table data

In [ ]:
launch_data = [['Launch_1', '2017-04-18', 8],
              ['Launch_2', '2017-07-08', 11],
              ['Launch_3', '2017-11-07', 20],
              ['Launch_4', '2018-05-06', 9],
              ['Launch_5', '2019-02-11', 8],
              ['Evergreen_1', '2018-02-13', 82],
              ['Evergreen_2', '2019-05-03', None]]
sql_stmt = 'INSERT INTO launch_info (launch, start, duration) VALUES (%s, %s, %s)'
cursor.executemany(sql_stmt, launch_data)

In [ ]:
# Launch 6
cursor.e('INSERT INTO launch_info (launch, start, duration) VALUES (%s, %s, %s)', values=['Launch_6', '2019-07-17', 12])

In [ ]:
conn.commit()

### Add launch subscriber data

In [ ]:
cursor.quick_info('launch_data')

#### Launch 6

In [ ]:
### Logic for if someone witnessed a launch (Use the tag)
launch_6_subs = get_api_data(CK_Api.TAGS, 1002191)

In [ ]:
len(launch_6_subs), launch_6_subs[0]['subscriber']

In [ ]:
values = [[l['subscriber']['id'], "Launch_6"] for l in launch_6_subs]; len(values), values[:3]

In [ ]:
cursor.e('INSERT INTO launch_data (subscriber_id, launch) VALUES (%s, %s)', values=values, many=True)

In [ ]:
conn.commit()

#### Evergreen 2

In [ ]:
# Check EVG 2 (4148 subs in CK)
len(cursor.e('SELECT * FROM launch_data l INNER JOIN subscriber_data s ON s.id=l.subscriber_id WHERE l.launch="Evergreen_2" AND s.state="active"'))

In [ ]:
# Get all Evergreen_2
cursor.execute('SELECT * FROM subscriber_data'); ck_data = {c['email_address']:c['id'] for c in cursor.fetchall()}
df = pd.read_csv("/Users/nish/Downloads/2019-06-05-32281.csv"); df.head(2)
tag_emails = df['email'].tolist()
# ck_inactive = [c.lower() for c in df[df['status']=='inactive']['email'].tolist()]; len(ck_inactive), ck_inactive[:3]

In [ ]:
values = [[ck_data[d], 'Evergreen_2'] for d in tag_emails]; values

In [ ]:
cursor.executemany('INSERT INTO launch_data (id, launch) VALUES (%s, %s)', values)

In [ ]:
conn.commit()

#### Other

In [ ]:
# Get the API dict too
ck_api_data = get_all_sub_data(columns=[])
ck_api_dict = {r[2]:r[3] for r in ck_api_data}

In [ ]:
# Get the email address from rds if their state doesn't match the csv's state
df = pd.read_csv("/Users/nish/Downloads/2019-06-04-30452.csv")
ck_dict = {e[0]:e[1] for e in df[['email', 'status']].to_numpy()}; len(ck_dict)

# Get RDS info
sql_stmt = "SELECT * FROM subscriber_data"
cursor.execute(sql_stmt); subscribers = cursor.fetchall(); len(subscribers)
rds_subscribers = [[s['id'], s['email_address'], s['state']] for s in subscribers]; rds_subscribers[:3]

# Find issues
diff = [[*r, ck_dict[r[1]]] for r in rds_subscribers if r[1] in ck_dict and ck_dict[r[1]]!=r[2]]; diff
# id, email, rds status, csv status

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE state="inactive"');
rds_inactive = [c['email_address'].lower() for c in cursor.fetchall()]; len(rds_inactive), rds_inactive[:3]

In [ ]:
# cursor.quick_info(email_address="mitsukai424@yahoo.com")
# cursor.quick_info(id=460778252)
# cursor.quick_info(state="inactive")
cursor.quick_info('purchase_data', subscriber_id=5197)
# cursor.quick_info('launch_data')
# cursor.quick_info('launch_data', launch="Launch_1")

In [ ]:
# See if any of the people labeled as "inactive" should actually be active
# and then remove all their subscribe dates otherwise

In [ ]:
conn.commit()

In [ ]:
# cursor.execute('UPDATE subscriber_data SET subscribe_date=NULL WHERE state="inactive"')

#### Misc issues

In [ ]:
# cursor.quick_info('purchase_data')
# cursor.quick_info('launch_data') #, id='8103'
# cursor.quick_info(email_address='@gmail.com')
# cursor.quick_info('purchase_data', purchase_email_address='jenslutsky@comcast.net')
# cursor.quick_info(id=7985)
# cursor.execute("SELECT max(id) FROM subscriber_data WHERE id < 20000"); cursor.fetchall()
# cursor.execute('SELECT * FROM purchase_data WHERE purchase_date="2017-11-25"'); cursor.fetchall()
# cursor.execute('SELECT * FROM purchase_data p INNER JOIN subscriber_data s ON p.subscriber_email_address=s.email_address')
# data = cursor.fetchall(); data
# [d for d in data if d['subscriber_id']!=d['id']]

In [ ]:
# Commands to update the databases
# cursor.execute('UPDATE subscriber_data SET subscribe_date="2019-05-10" WHERE email_address="cjgrrl@gmail.com"')
# cursor.execute('UPDATE subscriber_data SET id=7985 WHERE id=8103')
# cursor.execute('UPDATE purchase_data SET subscriber_email_address="jen@byjen.com", purchaser_id=NULL WHERE subscriber_id=137632941')

## Emails

#### Create and fill Database

In [ ]:
# Create a table for the emails
cursor.execute("""
    CREATE TABLE `email_data` (
      `subscriber_id` INT UNSIGNED NOT NULL,
      `email_id` INT UNSIGNED NOT NULL,
      `email_name` TEXT,
      `delivered` BOOLEAN,
      `opened` BOOLEAN,
      `clicked` BOOLEAN,
      `deliver_date` DATETIME,
      `open_date` DATETIME,
      `click_date` DATETIME,
      PRIMARY KEY (`subscriber_id`, `email_id`));"""
    )

In [ ]:
cursor.execute('ALTER TABLE `email_data` ADD COLUMN failed BOOLEAN AFTER bounced;')

In [ ]:
# Create a table for the emails
cursor.execute("""
    CREATE TABLE `email_data_freshness` (
      `subscriber_id` INT UNSIGNED NOT NULL,
      `last_checked` DATETIME,
      PRIMARY KEY (`subscriber_id`));"""
    )

In [ ]:
# cursor.execute('SHOW TABLES;'); cursor.fetchall()
# cursor.quick_info('email_data')

In [ ]:
cursor.execute

In [ ]:
# Get the emails for each subscriber
cursor.execute('SELECT * FROM subscriber_data WHERE id > 20000')
subscribers = cursor.fetchall()

# load the table with the email info

In [ ]:
session = get_ck_session()
for i, subscriber in enumerate(subscribers):
    if i < 10101: continue
    print(f'loading subscriber {i} / {len(subscribers)}', end='\r')
    try:
        subscriber['email_data'] = scrape_email_list(subscriber['id'], ck_app_session=session)
    except KeyboardInterrupt as k:
        break
    except ConnectionError as c:
        print(f'Possible subscriber deletion on {i}, {subscriber["id"]}') 
    except:
        print(f'Failed on subscriber {i}, {subscriber["id"]}')

In [ ]:
columns = [
    'subscriber_id', 
    'email_id', 
    'email_name', 
    'delivered', 
    'opened', 
    'clicked', 
    'bounced',
    'failed',
    'deliver_date', 
    'open_date', 
    'click_date'
]
values = []
values = [[s['id'], *[e[c] for c in columns[1:]]] for s in subscribers[10100:] if 'email_data' in s for e in s['email_data'] ]

column_string = ", ".join(["`"+col+"`" for col in columns]); column_string
variable_string = ", ".join(["%s"]*len(columns)); variable_string
duplicate_string = "ON DUPLICATE KEY UPDATE "+", ".join(["`"+col+"`=VALUES(`"+col+"`)" for col in columns]); duplicate_string

# # Prep statement
sql_string = f'insert into email_data ({column_string}) values ({variable_string}) {duplicate_string};'; sql_string

# # Add those subscribers to the database
cursor.executemany(sql_string, values)

In [ ]:
conn.commit()

In [ ]:
cursor.quick_info('email_data', email_id=272065)
# cursor.execute('SELECT * FROM email_data WHERE deliver_date > "2018-09-21" AND deliver_date <= "2018-09-30"')

In [ ]:
cursor.execute('DELETE FROM email_data')

#### Update freshness table

In [ ]:
sql_stmt = """\
SELECT e.* FROM email_data e
INNER JOIN subscriber_data s
    ON e.subscriber_id = s.id
WHERE s.state != "active"
LIMIT 100
"""
cursor.execute(sql_stmt)

In [ ]:
cursor.fetchall()

#### Run refill 

In [ ]:
cursor.quick_info('email_data_freshness')#, subscriber_id=3)

In [ ]:
# cursor.quick_info('email_data_freshness')#, subscriber_id=3)
sql_stmt = """\
SELECT s.id, e.last_checked 
FROM email_data_freshness e 
LEFT JOIN subscriber_data s
    ON s.id = e.subscriber_id
WHERE 
    last_checked > NOW() - INTERVAL 4 HOUR - INTERVAL 2 HOUR
    AND s.state = "active"
"""
cursor.execute(sql_stmt)
a = cursor.fetchall(); len(a)

In [ ]:
logging.getLogger().setLevel(logging.INFO)
scrape_all_email_data(conn, exclude_freshness=2)

In [ ]:
notify('success')

## Inactives

In [ ]:
update_rds_with_inactives(conn, only_new=True)#limit_to_past_days=30)

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE state="inactive"')
# Should match to count from https://app.convertkit.com/?status=inactive

In [ ]:
conn.commit()

## Error Checks

#### RDS subscriber data versus the Web

##### Get data

In [ ]:
# RDS Data
# conn.commit()
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

In [ ]:
log_level('w')

In [ ]:
# Takes about 10 seconds
all_ck_list = get_api_data(CK_Api.ALL_SUBS, None, session=session)
all_ck_dict = {a['id']: a for a in all_ck_list}

print(all_rds_list[0]); print(); print(all_ck_list[0])

##### Summarize Issues

In [ ]:
# Duplicate emails in RDS
duplicate_rds_emails = [c for c in Counter([a['email_address'] for a in all_rds_list]).items() if c[1]>1]
print(f'Duplicate emails in RDS: {len(duplicate_rds_emails)}')

# Total Mismatches
mismatches = [a for a in all_rds_dict if a in all_ck_dict and array_mismatch(all_rds_dict[a], all_ck_dict[a])]
print(f'Total mismatches: {len(mismatches)}')

print(f'There are {len(set([a["lead_source"] for a in all_rds_list]))} / 10 lead_sources')
print(f'There are {len(set([a["lead_opt-in"] for a in all_rds_list]))} / 14 lead_opt-ins')

# See if any subscribers are missing from RDS
## Get step function data first
get_id = lambda x: int(re.match(r'.*?_(\d+)', x).group(1))
recent_execution = lambda x: x['stopDate'] > datetime.now().astimezone(pytz.timezone('US/Eastern')) - timedelta(hours=1)
client = boto3.client('stepfunctions')
sub_arn = 'arn:aws:states:us-east-1:787744691769:stateMachine:CK_Subscription'
running_executions = client.list_executions(stateMachineArn=sub_arn, statusFilter='RUNNING')['executions']
success_executions = client.list_executions(stateMachineArn=sub_arn, statusFilter='SUCCEEDED')['executions']
executions = running_executions + [e for e in success_executions if recent_execution(e)]
processing_subs = [get_id(e['name']) for e in executions]; len(processing_subs), processing_subs

rds_missing_subs = [a for a in all_ck_dict if a not in all_rds_dict and a not in processing_subs]
print(f'CK subs missing from RDS: {len(rds_missing_subs)}')

# These are probably ok, but are delayed due to delay of gathering ck data
today = datetime.strftime(datetime.now(), '%Y-%m-%d')
rds_not_in_ck_subs = [a for a in all_rds_dict if all_rds_dict[a]['state'] == 'active' and a not in all_ck_dict and all_rds_dict[a]['subscribe_date'] != today]
print(f'RDS subs missing from CK: {len(rds_not_in_ck_subs)}')

# Mismatch between email_data and email_data_freshness (active subscribers with email_data but no email_data_freshness)
sql_stmt = """\
SELECT s.id FROM email_data e
LEFT JOIN subscriber_data s
    ON s.id = e.subscriber_id
LEFT JOIN email_data_freshness f
    ON f.subscriber_id = e.subscriber_id
WHERE f.subscriber_id IS NULL
    AND s.state = "active"
"""
cursor.execute(sql_stmt)
missing_freshness_ids = [c['id'] for c in cursor.fetchall()]; len(missing_freshness_ids), missing_freshness_ids[:3]
print(f'Missing freshness data: {len(missing_freshness_ids)}')

# Subscribers with no email data
sql_stmt = """\
SELECT s.id FROM subscriber_data s
LEFT JOIN email_data e
    ON e.subscriber_id = s.id
WHERE e.subscriber_id IS NULL
    AND s.state = "active"
"""
cursor.execute(sql_stmt)
no_email_subscribers = [c['id'] for c in cursor.fetchall()]; len(no_email_subscribers), no_email_subscribers[-3:]
print(f'Subscribers with no email data: {len(no_email_subscribers)}')

# Subscriber with no subscribe date
no_subscribe_date_rds = [a['id'] for a in all_rds_list if a['state'] == 'active' and not a['subscribe_date']]
print(f'RDS Subscribers with no subscribe_date: {len(no_subscribe_date_rds)}')

no_subscribe_date_ck = [a['id'] for a in all_ck_list if a['state'] == 'active' and not a['subscribe_date']]
print(f'CK Subscribers with no subscribe_date: {len(no_subscribe_date_ck)}')

# active subscriber with an unsubscribe_date (resubscriber)
err_unsub_date = [a['id'] for a in all_rds_list if a['state'] == 'active' and a['unsubscribe_date']]
print(f'Unnecessary unsub dates: {len(err_unsub_date)}')


In [ ]:
conn.commit()

In [ ]:
# REFRESH CK_SUB DATA 
# Update the changed ck_subs
new_items = []
changed_ck_subs = err_unsub_date #Out[57]# mismatches#err_unsub_date #rds_not_in_ck_subs + no_subscribe_date_rds
# Remove the items from the list
# type(a) == dict to avoid string issues
all_ck_list = [a for a in all_ck_list if type(a)==dict and a['id'] not in changed_ck_subs]
# Get the new items
for i, updated_id in enumerate(changed_ck_subs):
    try:
        new_items.append(get_api_data(CK_Api.SUB, updated_id, session=session))
    except SubscriberDNE as s: pass
    print(f'Updating {i}/{len(changed_ck_subs)} ', end='\r')
# Add those to the list
all_ck_list += new_items
# Update the dictionary
for item in new_items:
    all_ck_dict[item['id']] = item

##### Temp analysis

In [ ]:
no_subscribe_date_ck

In [ ]:
rds_not_in_ck_subs

In [ ]:
for s in no_subscribe_date_ck:
    if s not in no_subscribe_date_rds:
        update_single_subscriber(s, cursor, geolocator = Nominatim(user_agent='dgn_app'))

In [ ]:
temp_id = mismatches[0]
temp = array_mismatch(all_rds_dict[temp_id], all_ck_dict[temp_id]); temp

In [ ]:
str(temp['loc_lat'][0]) == temp['loc_lat'][1]

In [ ]:
conn.commit()

##### Fix duplicate email addresses in RDS

In [ ]:
# Delete duplicates from RDS
for duplicate_email, count in duplicate_rds_emails:
    print(duplicate_email)
    duplicate_ids = [a['id'] for a in cursor.e(f'SELECT id FROM subscriber_data WHERE email_address=%s', values=[duplicate_email])]
    print(duplicate_ids)
    for duplicate_id in duplicate_ids:
        try:
            get_single_subscriber(subscriber_id=duplicate_id)
        except SubscriberDNE as s:
            cursor.e(f'DELETE FROM subscriber_data WHERE id={duplicate_id}')
            print(f'deleted subscriber {duplicate_id} from RDS')

In [ ]:
# Update launch data!
# cursor.quick_info('launch_data', subscriber_id=7561)
cursor.e('UPDATE launch_data SET subscriber_id=314547030 WHERE subscriber_id=7561')

In [ ]:
# Duplicate emails in RDS
duplicate_rds_emails = [c for c in Counter([a['email_address'] for a in all_rds_list]).items() if c[1]>1]
print(f'Duplicate emails in RDS: {len(duplicate_rds_emails)}')
duplicate_rds_emails[:3]

In [ ]:
cursor.e('SELECT * FROM subscriber_data WHERE email_address="dnishiyama+test2@gmail.com"')
# cursor.e('DELETE FROM subscriber_data WHERE id=8468')

In [ ]:
delete_ids = []
for duplicate_email, count in duplicate_rds_emails:
    cursor.execute(f'SELECT id, state FROM subscriber_data WHERE email_address = "{duplicate_email}"')
    sub_data = cursor.fetchall()
    if len(sub_data) != 2: raise Exception() # Otherwise this isn't a duplicate!
    id_0 = sub_data[0]['id']; id_1 = sub_data[1]['id']
    
    if id_0 > 20000: # deletion is no problem if not real subscriber (id < 20000)
        # If it is a real subscriber (id > 20000), then deletion is a problem only if the state isn't deleted 
        if sub_data[0]['state'] != 'deleted': # deletion is no problem if it is deleted
            if sub_data[1]['state'] == 'deleted':  # or switch ids if this one is deleted
                id_0 = id_1
                id_1 = sub_data[0]['id']
                print(f'switched ids on {duplicate_email} due to one being deleted and not the other')
            else:
                print(f'continuing on {duplicate_email} due to high id_0 and not a deletion')
                continue
        else: pass
    else: pass
        
    if sub_data[0]['state'] == 'active': raise Exception()
    if id_1 < 20000: continue
    if cursor.execute(f'SELECT * FROM launch_data WHERE subscriber_id = {id_0}') > 0:
        print(f'updating launch_data for {id_0} to {id_1}')
        cursor.execute(f'UPDATE launch_data SET subscriber_id={id_1} WHERE subscriber_id={id_0}')

    if cursor.execute(f'SELECT * FROM purchase_data WHERE subscriber_id = {id_0}') > 0:
        print(f'updating purchase_data, subscriber_id for {id_0} to {id_1}')
        cursor.execute(f'UPDATE purchase_data SET subscriber_id={id_1} WHERE subscriber_id={id_0}')

    if cursor.execute(f'SELECT * FROM purchase_data WHERE purchase_id = {id_0}') > 0:
        print(f'updating purchase_data, purchase_id for {id_0} to {id_1}')
        cursor.execute(f'UPDATE purchase_data SET purchase_id={id_1} WHERE purchase_id={id_0}')
    print(f'ready to delete {id_0}', 'for', duplicate_email )
    delete_ids.append(id_0)


In [ ]:
cursor.execute(f'DELETE FROM subscriber_data WHERE id in {mysql_array(delete_ids)}')

In [ ]:
# Refresh RDS and CK data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

##### Bad Sources and Optins

In [ ]:
[a['id'] for a in all_ck_list if a['lead_opt-in'] == None]

In [ ]:
set([a['lead_opt-in'] for a in all_ck_list])

In [ ]:
set([a['lead_source'] for a in all_ck_list])

In [ ]:
# Fix lead source by going to each with that CK data and updating, then updating the rds data
try:
    bad_subs = [a for a in all_ck_list if a['lead_source'] in ['', 'FB Ad?fbclid', 'FB+Ad', 'Medium?source',None,'None']]; 
#     print(len(bad_subs), bad_subs[:1])
    bad_subs += [a for a in all_ck_list if a['lead_opt-in'] in ['', 'None', 'style mini-course', None]]; 
#     len(bad_subs), bad_subs[:3]
    for i, a in enumerate(bad_subs):
        new_source = {}; new_optin = {};
        if a['lead_source'] in ['',None,'None']: new_source = {'lead_source': 'Unknown'}
        if a['lead_source'] in ['FB Ad?fbclid', 'FB+Ad']: new_source = {'lead_source': 'FB Ad'}
        if a['lead_source'] in ['Medium?source']: new_source = {'lead_source': 'Medium'}
        if new_source: print(f'Updating {a["id"]} with source {new_source} instead of {a["lead_source"]}')

        if a['lead_opt-in'] in ['', None]: new_optin = {'lead_opt-in': 'Unknown'}
        if a['lead_opt-in'] in ['style mini-course']: new_optin = {'lead_opt-in': 'Style Mini-course'}
        if new_optin: print(f'Updating {a["id"]} with optin {new_optin} instead of {a["lead_opt-in"]}')

        update_ck_sub_with_misc(a["id"], session=session, **{**new_source, **new_optin})
        input('continue?'); clear_output()
except KeyboardInterrupt as k:
    pass


In [ ]:
cursor.e('UPDATE subscriber_data SET `lead_opt-in` = "Unknown" WHERE `lead_opt-in`="" OR `lead_opt-in` IS NULL')

In [ ]:
bad_optin = [["Style Mini-course", a['id']] for a in all_rds_list if a['lead_opt-in'] in ['style mini-course']]; len(bad_optin), bad_optin[:3]
cursor.executemany('UPDATE subscriber_data SET `lead_opt-in`=%s WHERE id=%s', bad_optin)

In [ ]:
# Fix Unknown Opt-ins
bad_optin = [["Unknown", a['id']] for a in all_rds_list if a['lead_opt-in'] in ['', None, 'None']]; len(bad_optin), bad_optin[:3]
# cursor.executemany('UPDATE subscriber_data SET `lead_opt-in`=%s WHERE id=%s', bad_optin)

In [ ]:
# Fix FB Ads
bad_source = [['FB Ad', a['id']] for a in all_rds_list if a['lead_source'] in ['FB Ad?fbclid', 'FB+Ad']]; len(bad_source), bad_source[:3]
cursor.executemany('UPDATE subscriber_data SET lead_source=%s WHERE id=%s', bad_source)
# for _, bad_id in bad_source:
#     update_ck_sub_with_misc(sub_id=bad_id, session=session, lead_source="FB Ad")

In [ ]:
list(range(1820,len(bad_source),500))

In [ ]:
# Fix Unknowns
steps=250; start = 0; # normally 0
bad_source = [['Unknown', a['id']] for a in all_rds_list if a['lead_opt-in'] in ['None', None, '']]; len(bad_source), bad_source[:3]
for i in range(start, len(bad_source), steps):
    print(f"Updating data from {i} to {i+steps} out of {len(bad_source)}", end='\r')
    cursor.executemany('UPDATE subscriber_data SET lead_source=%s WHERE id=%s', bad_source[i:i+steps])

In [ ]:
conn.commit()

In [ ]:
for i, a in enumerate(mismatches):
    if 'lead_source' in array_mismatch(all_rds_dict[a], all_ck_dict[a]):
        print(f'Updating {a} with {all_rds_dict[a]["lead_source"]} instead of {all_ck_dict[a]["lead_source"]}')
        update_ck_sub_with_misc(a, session=session, **{"lead_source": all_rds_dict[a]["lead_source"]})
        

##### Fix misc mismatches

In [ ]:
len(mismatches), mismatches[:3]

In [ ]:
cancellers = get_api_data(CK_Api.ALL_SUBS, None, include_unsubscribers=True)

In [ ]:
[c for c in cancellers if 

In [ ]:
# Fix mismatches of loc_lat and loc_lng of None (rds) vs 'null' (ck)

In [ ]:
ok_columns = [
    'stsbc_evg_launch_deadline',
    'other_email_address',
    'lead_source',
    'lead_opt-in',
    'lead_source',
    'state',
    'subscribe_date',
    'unsubscribe_date',
    'loc_lat',
    'loc_lng',
    'loc_city',
    'loc_state',
]
for mismatch in mismatches:
    data = array_mismatch(all_rds_dict[mismatch], all_ck_dict[mismatch])
    if all(d in ok_columns for d in data.keys()): continue
    print(mismatch, data)
    input()
#     clear_output()

In [ ]:
# Fix some issues 
temp_ids = [a['id'] for a in all_ck_list if a['lead_opt-in'] and a['lead_opt-in'].lower()==a['lead_opt-in']]
[all_ck_dict[t]['lead_opt-in'] for t in temp_ids]
fields = {'lead_opt-in': 'Emails'}
for temp_id in temp_ids:
    update_ck_sub_with_misc(temp_id, session=session, **fields)

In [ ]:
# Manual choice for each item
try:
    for i, a in enumerate(mismatches):
#         if i < 567: continue
        print(f'Updating {i} / {len(mismatches)}', end='\r')
        print('RDS:', all_rds_dict[a])
        print('CK:', all_ck_dict[a])
        print('\n', i, a,  array_mismatch(all_rds_dict[a], all_ck_dict[a]) )
        cont = input('KEEP the RDS or CK values? RDS/(CK)')
        if cont == 'CK' or cont == '':
            update_subscriber_in_rds(a, cursor, session=session)
        elif cont == 'RDS':
            fields = {k:all_rds_dict[a][k] for k in array_mismatch(all_rds_dict[a], all_ck_dict[a]).keys()}
            update_ck_sub_with_misc(a, session=session, **fields)
        clear_output()
except KeyboardInterrupt as k:
    pass # Allow keyboard break

In [ ]:
# Load all the CK data into RDS
for i, a in enumerate(mismatches):
    print(f'Updating {i} / {len(mismatches)}', end='\r')
    update_subscriber_in_rds(a, cursor, sub_data=all_ck_dict[a])
#     elif cont == 'RDS':
#         fields = {k:all_rds_dict[a][k] for k in array_mismatch(all_rds_dict[a], all_ck_dict[a]).keys()}
#         update_ck_sub_with_misc(a, session=session, **fields)

In [ ]:
conn.commit()

##### Fix RDS subs that are missing from the active CK list

In [ ]:
a = rds_not_in_ck_subs[0]
array_mismatch(all_rds_dict[a], all_ck_dict[a])

In [ ]:
get_api_data(CK_Api.SUB, 137389801, session=session)

In [ ]:
log_level('i')
# log_level('d')
ck_app_session = get_ck_session()

In [ ]:
cursor.e('SELECT DISTINCT state FROM subscriber_data')

In [ ]:
rds_not_in_ck_subs[104:] #.index(469283448)

In [ ]:
# Diagnose why it is in RDS, but not CK
# sub_id = 496615911
ck_app_session=get_ck_session()
for sub_id in rds_not_in_ck_subs[104:]:
    try:
        sub_data = get_api_data(CK_Api.SUB, sub_id, session=session)
#         if sub_data['state'] != 'active':
        print(f'Subscriber {sub_id}, {sub_data}', end='\r')
        if sub_data['state'] == 'cancelled':
            manual_ck_unsub_update(sub_id, cursor, ck_app_session=ck_app_session, session=session)
        elif sub_data['state'] in ['bounced', 'complained']:
            state = sub_data['state']
            cursor.e(f'UPDATE subscriber_data SET state=%s WHERE id={sub_id}', state)
        else:
            input()
    except SubscriberDNE as s:
#         print(f'Subscriber {sub_id} deleted')
        cursor.e(f'UPDATE subscriber_data SET state="deleted" WHERE id={sub_id}')
    

In [ ]:
conn.commit()

In [ ]:
# All active RDS should be in CK list, if not they need to be updated
ck_missing_subs = [a for a in all_rds_list if a['state']=="active" and a['id'] not in all_ck_dict]; len(ck_missing_subs), ck_missing_subs[:3]

In [ ]:
for i, ck_missing_sub in enumerate(ck_missing_subs):
    print(f'{len(ck_missing_subs) - i} remaining')
    ck_missing_id = ck_missing_sub["id"] # Get the id
    print(f'working on {ck_missing_id}')
    print(get_json_columns(ck_missing_sub, ["id","state", "subscribe_date", "unsubscribe_date"]))
    try:
        scraped_ck_data = get_single_subscriber(subscriber_id=ck_missing_id)
        print(get_json_columns(scraped_ck_data, ["id","state", "subscribe_date", "unsubscribe_date"]))
        new_state = scraped_ck_data["state"]
    except SubscriberDNE as s:
        print('subscriber_DNE')
        new_state = "deleted"
        
    sql_stmt = f'UPDATE subscriber_data SET state="{new_state}" WHERE id = {ck_missing_id}'
    print(sql_stmt)
    updated_rows = cursor.execute(sql_stmt)
    ck_missing_subs[i]['state'] = new_state

In [ ]:
# Refresh RDS data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

##### Fix RDS subs that are unsubscribed with no unsubscribe date

In [ ]:
# Get unsubscribers with no unsubscribe_date
no_unsub_dates = [r for r in all_rds_list if r['state'] == 'cancelled' and not r['unsubscribe_date']]; 
len(no_unsub_dates), [get_json_columns(n, ['id', 'state', 'unsubscribe_date']) for n in no_unsub_dates[:3]]

In [ ]:
# Get the actual unsubscribe dates 
ck_app_session = get_ck_session()
new_unsub_data = []
for i, no_unsub_date in enumerate(no_unsub_dates):
    print(i, no_unsub_date['id'], end='\r')
    unsub_datetime = scrape_unsub_datetime(no_unsub_date['id'], ck_app_session=ck_app_session)
    unsub_date = datetime.strftime(unsub_datetime, '%Y-%m-%d')
    new_unsub_data.append([unsub_date, no_unsub_date['id']])

In [ ]:
# update RDS
cursor.executemany('UPDATE subscriber_data SET unsubscribe_date=%s WHERE id=%s', new_unsub_data)

In [ ]:
# update CK
for i, (unsub_date, unsub_id) in enumerate(new_unsub_data):
    print(f'Updating # {i} for {unsub_id}', end='\r')
    update_unsub_with_day(unsub_id, unsub_date)

In [ ]:
# Refresh RDS and CK data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

##### Fix missing subs from RDS (that are in CK)

In [ ]:
rds_missing_subs = [a for a in all_ck_dict if a not in all_rds_dict]
print(f'Missing subs from RDS: {len(rds_missing_subs)}, {" ".join([str(r) for r in rds_missing_subs[:3]])}...')

In [ ]:
for rds_missing_sub_id in rds_missing_subs:
    update_subscriber_in_rds(
        rds_missing_sub_id, 
        conn.cursor(), 
        sub_data=all_ck_dict[rds_missing_sub_id], 
        session=session
    )

In [ ]:
# Refresh RDS and CK data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

In [ ]:
logging.getLogger().setLevel(logging.INFO)
scrape_all_email_data(conn, exclude_freshness=1)

##### Fix incorrect States

In [ ]:
# subs in RDS that don't match the CK state (active)
incorrect_states_1 = [a for a in all_rds_dict if a in all_ck_dict and all_ck_dict[a]['state'] != all_rds_dict[a]['state']]

# subs that are active in RDS but not in CK
incorrect_states_2 = [a for a in all_rds_dict if all_rds_dict[a]['state'] == 'active' and a not in all_ck_dict]

In [ ]:
for i in incorrect_states_1:
    print(all_rds_dict[i])
    print()
    print(all_ck_dict[i])
    print()
    print()
    print()


In [ ]:
values = [[all_ck_dict[i]['state'], i] for i in incorrect_states_1]
cursor.executemany('UPDATE subscriber_data SET state = %s WHERE id = %s', values)

In [ ]:
conn.commit()

In [ ]:
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

##### unnecessary unsub date

In [ ]:
len(err_unsub_date), err_unsub_date[:3]

In [ ]:
for sub_id in err_unsub_date:
    update_ck_sub_with_misc(sub_id, session=session, unsubscribe_date=None)
    cursor.execute(f'UPDATE subscriber_data SET unsubscribe_date=NULL WHERE id={sub_id}')
conn.commit()

In [ ]:
# Legacy

In [ ]:
for i, sub_id in enumerate(err_unsub_date):
    print(f'Updating {i}, {sub_id}', end='\r')
    update_unsub_with_day(sub_id, day=None)
    cursor.execute(f'UPDATE subscriber_data SET unsubscribe_date=NULL WHERE id={sub_id}')

In [ ]:
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}


##### Missing Freshness Data

In [ ]:
for sub_id in missing_freshness_ids + no_email_subscribers:
    email_data = scrape_email_list(sub_id, ck_app_session)
    print(f'Got {len(email_data)} emails for {sub_id}. ', end=' ')
    rows = save_email_data(sub_id, email_data, conn)
    print(f'Wrote {rows} emails ')

##### Correct no subscribe_date

In [ ]:
no_subscribe_date_rds[0]
# update_single_subscriber??

In [ ]:
# Method as of 9/16/19
# nones = [None, 'null', ''];
# for sub_id in no_subscribe_date_rds:
#     sub_data = all_ck_dict[sub_id]
#     if sub_data['subscribe_date'] in nones:
#         if sub_data['state'] in ['active', 'cancelled']:
#             if 'mailchimp_sub_date' in sub_data and sub_data['mailchimp_sub_date']:
#                 subscribe_date = mailchimp_conv(sub_data['mailchimp_sub_date'])
#             else:
#                 subscribe_date = created_at_conv(sub_data['created_at'])
#         else: 
#             raise Exception('non-active and non-cancelled states not designed for yet')
#         break
#         sub_data['subscribe_date'] = subscribe_date

In [ ]:
# Method as of 8/3/19
for i, sub_id in enumerate(no_subscribe_date_rds):
    print(f'getting {i}/{len(no_subscribe_date_rds)}', end='\r')
    update_single_subscriber(
        sub_id, 
        cursor, 
        ck_app_session=ck_app_session, 
        do_location=False
    )

In [ ]:
# Legacy Method

In [ ]:
new_sub_data = []
for no_sub_date_id in no_subscribe_date_rds:
    new_sub_data.append([get_subscribe_date_from_email_click(no_sub_date_id, conn, force_date=True), no_sub_date_id])
new_sub_data

In [ ]:
cursor.executemany('UPDATE subscriber_data SET subscribe_date=%s WHERE id=%s', new_sub_data)

In [ ]:
# refresh RDS Data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

In [ ]:
for i, no_sub_date_id in enumerate(no_subscribe_date_ck):
    if no_sub_date_id not in all_rds_dict:
        print(f'Still waiting on data for {no_sub_date_id}')
        continue
    update_ck_sub_with_misc(no_sub_date_id, session=session, **{'subscribe_date':all_rds_dict[no_sub_date_id]['subscribe_date']})
    print(f'updated #{i}, {no_sub_date_id}', end='\r')

##### Other

In [ ]:
# Get step function execution subscribers to ignore
get_id = lambda x: int(re.match(r'.*?_(\d+)', x).group(1))
recent_execution = lambda x: x['stopDate'] > datetime.now().astimezone(pytz.timezone('US/Eastern')) - timedelta(hours=1)
client = boto3.client('stepfunctions')
sub_arn = 'arn:aws:states:us-east-1:787744691769:stateMachine:CK_Subscription'
running_executions = client.list_executions(stateMachineArn=sub_arn, statusFilter='RUNNING')['executions']
success_executions = client.list_executions(stateMachineArn=sub_arn, statusFilter='SUCCEEDED')['executions']
executions = running_executions + [e for e in success_executions if recent_execution(e)]
processing_subs = [get_id(e['name']) for e in executions]; len(processing_subs), processing_subs

In [ ]:
sub_id = re.match(r'.*?_(\d+)(_.*?)?', 'a_517497884').group(1)

In [ ]:
[e['name'] for e in executions['executions'] if e['stopDate'] > datetime.now().astimezone(pytz.timezone('US/Eastern')) - timedelta(hours=1)]

In [ ]:
processing_subs

In [ ]:
rds_missing_subs

In [ ]:
conn.commit()

In [ ]:
# Add subscriber data to RDS
for sub_id in missing_subs:
    if email in processing_subs: continue

    sub_id = get_single_subscriber(email_address=email, session=session)['id']
    print(sub_id)
    print(update_single_subscriber(sub_id, conn))
    input("continue?")

In [ ]:
conn.commit()

In [ ]:
cursor.quick_info(email_address='eurilaann.gonzales@yahoo.com')

#### Correct Launch Data for no-subscriber Purchases

In [ ]:
# Evaluate and correct launch names
# cursor.e('SELECT * FROM purchase_data WHERE launch="Giveaway"')
# cursor.e('UPDATE purchase_data SET purchase_value=0 WHERE launch="Giveaway"')
# cursor.e('UPDATE IGNORE launch_data SET launch="Giveaway" WHERE launch="Unknown"')

In [ ]:
# When people purchase from a new email address, they don't have any information.

In [ ]:
purchasers = set([a for b in cursor.e('SELECT subscriber_id, purchaser_id, launch FROM purchase_data') for a in ((b['launch'] if b['launch'] else 'Unknown', b['subscriber_id']), (b['launch'] if b['launch'] else 'Unknown',b['purchaser_id']))])
values = [p for p in purchasers if p[1]]; len(values), values[:2]

In [ ]:
sql_stmt = """INSERT IGNORE INTO launch_data (`launch`, `subscriber_id`) VALUES (%s, %s)"""
updated_rows = cursor.e(sql_stmt, values=values, many=True)

In [ ]:
updated_rows

#### Incorrect optins (based on email and forms)

##### Get all the data needed

In [ ]:
# RDS Data
cursor.execute('SELECT * FROM subscriber_data'); 
all_rds_list = cursor.fetchall()
all_rds_dict = {a['id']: a for a in all_rds_list}

In [ ]:
sql_stmt = """\
SELECT e.subscriber_id, e.email_id, e.email_name, e.deliver_date, e.open_date, e.click_date
FROM email_data e 
"""
mysql_email_data = cursor.e(sql_stmt); len(mysql_email_data)

In [ ]:
# Go through all emails. If it is the first or min, update that item in subscriber dict, otherwise move on
email_data = {}
def min_date(x):
    dates=['deliver_date', 'click_date', 'open_date']
    for date in dates:
        x[date] = x[date] if x[date] else datetime.now()
    return min(x[date] for date in dates)
    
for i, m in enumerate(mysql_email_data):
    if m['subscriber_id'] not in email_data or min_date(m) < min_date(email_data[m['subscriber_id']]):
        email_data[m['subscriber_id']] = m

In [ ]:
# Add the optins
for k in email_data:
    if k not in all_rds_dict: 
        print(f'issue with {k}')
        continue
    email_data[k]['lead_opt-in'] = all_rds_dict[k]['lead_opt-in']

In [ ]:
email_data[180021887]

##### Make updates

In [ ]:
set([a["lead_opt-in"] for a in all_rds_list])

In [ ]:
# Make a list of acceptable pairings; if not in, confirm, then add to or mark as not ok
# ok_pairings = []
# fixed = []
# ignore = []

comp = lambda x: set((o['email_id'], o['lead_opt-in']) for o in x)
comp_only_email_id = lambda x: set(o['email_id'] for o in x)

for i, email_datum in enumerate(email_data.values()): # We have it saved and it is right already
    if i < 19744: continue #shortcut
    if 'lead_opt-in' not in email_datum: continue
    print(f'Processing {i} / {len(email_data.values())}, fixed {len(fixed)}', end='\r')
    if (email_datum['email_id'], email_datum['lead_opt-in']) in comp(ok_pairings):
        # No fix
        if email_datum in ok_pairings: continue
        ok_pairings.append(email_datum)
    elif (email_datum['email_id'] in comp_only_email_id(ok_pairings)): #we have it saved, automatic fix
        # automatic fix
        correct_opt_in = [e for e in ok_pairings if e['email_id'] == email_datum['email_id']][0]['lead_opt-in']
        email_datum['lead_opt-in'] = correct_opt_in
        cursor.e('UPDATE subscriber_data SET `lead_opt-in`=%s WHERE id=%s', values=[correct_opt_in, email_datum['subscriber_id']])
        fixed.append(email_datum)
    else:
        # Manual fix
        print(f'Processing {i} / {len(email_data.values())}')
        print(f'ok_pairings: {len(ok_pairings)}, fixed: {len(fixed)}')
        print(email_datum)
        correct_opt_in = input('What is the corrent pairing? Blank if it is correct: ')
        if correct_opt_in == 'ignore':
            clear_output()
            continue
        elif correct_opt_in: 
            email_datum['lead_opt-in'] = correct_opt_in
            cursor.e('UPDATE subscriber_data SET `lead_opt-in`=%s WHERE id=%s', values=[correct_opt_in, email_datum['subscriber_id']])
        ok_pairings.append(email_datum)
        clear_output()
        

In [ ]:
list(enumerate(email_data.values()))[19744]

In [ ]:
ok_pairings: 9496, fixed: 6555

In [ ]:
# Next go through unknown and see if they signed up for a form

In [ ]:
d = get_api_data(CK_Api.FORM_SUBS, 341476, session=session)
len(d)

In [ ]:
295490: sketchbook mini-course
975316: special case for ebook (155418278)
1024896: Sketchbook to Style Boot Camp
750878: MCDT
341476: MCDT
1139190: MCDT
sub: 180021887 Something is up with early incentive emails
387829: studiomates waitlist

In [ ]:
email_data_temp = cursor.e('SELECT * FROM email_data e WHERE subscriber_id=137632701');email_data_temp[0]


In [ ]:
min(a['deliver_date'] for a in email_data_temp)

In [ ]:
# Assume that sequences continue and that tuesday broadcasts go out

##### Update CK

In [ ]:
# Get mismatches (only for lead_opt-in)
# Only get the lead opt-in. If you get everything, you might make unnecessary update
mismatches = [a for a in all_rds_dict if a in all_ck_dict and array_mismatch(all_rds_dict[a], all_ck_dict[a]) and 'lead_opt-in' in array_mismatch(all_rds_dict[a], all_ck_dict[a])]
len(mismatches)

In [ ]:
try:
    for i, a in enumerate(mismatches):
#         if i < 567: continue
        print(f'Updating {i} / {len(mismatches)}', end='\r')
        fields = {k:all_rds_dict[a][k] for k in array_mismatch(all_rds_dict[a], all_ck_dict[a]).keys()}
        update_ck_sub_with_misc(a, session=session, **fields)
#         clear_output()
except KeyboardInterrupt as k:
    pass # Allow keyboard break

##### Check results

In [ ]:
mismatches = [a for a in all_rds_dict if a in all_ck_dict and array_mismatch(all_rds_dict[a], all_ck_dict[a])]
print(f'Total mismatches: {len(mismatches)}')

In [ ]:
mismatches[800]

#### Unknown Optins (MailChimp also)

##### Using Forms

In [ ]:
unknowns = cursor.e('SELECT * FROM subscriber_data WHERE `lead_opt-in`="Unknown"'); len(unknowns)

In [ ]:
list(email_data.keys())[:3]

In [ ]:
# Get all the form data, see if they signed up for a form at similar time to first email
forms = {g['id']: g['name'] for g in get_api_data(CK_Api.FORM_LIST, None, session=session)}; forms

In [ ]:
form_subs = {}
for form in forms.items():
    form_subs[form[0]] = get_api_data(CK_Api.FORM_SUBS, form[0], session=session)

In [ ]:
# Need to know, for each unknown, if there was a form that they joined near when they got their first email

In [ ]:
# Get the earliest form for each subscriber
sub_forms = {}
for i, (form_id, form_subscriptions) in enumerate(form_subs.items()):
    print(f'Processing {i} / {len(form_subs)}', end='\r')
    for form_subscription in form_subscriptions:
        sub_id = form_subscription['subscriber']['id']
        if sub_id not in [u['id'] for u in unknowns]: continue
        created_at = created_to_dt(form_subscription['created_at'])
        if created_at < sub_forms.get(sub_id, {}).get('dt', datetime.now()):
            sub_data = {
                'form': form_id, 
                'dt': created_to_dt(form_subscription['created_at']),
                'sub_create': created_to_dt(form_subscription['subscriber']['created_at'])
            }
            sub_forms[sub_id] = sub_data

In [ ]:
deltas = []
for k,v in sub_forms.items():
    deltas.append(v['dt']- v['sub_create'])
deltas = [d.total_seconds()/60/60/24 for d in deltas if d.total_seconds()/60/60/24<100]
plt.plot(deltas)

In [ ]:
set((v['form'], forms[v['form']]) for v in sub_forms.values())

In [ ]:
{'Book Making',
 'Emails',
 'Influence Map',
 'MCDT',
 'Mailchimp',
 'Mini-course',
 None,
 'Sketchbook Mini-course',
 'StSBC WL',
 'Studiomates WL',
 'Style Mini-course',
 'Tools',
 'Unknown',
 'WAF Activity Guide',
 'WAF ebook'}

In [ ]:
forms_key = {
    272065: 'Tools',
    272096: 'WAF ebook',
    272097: 'Book Making',
    295490: 'Sketchbook Mini-course',
    330122: 'StSBC WL',
    341476: 'MCDT',
    456516: 'Influence Map',
    458062: 'Influence Map',
    459136: 'Style Mini-course',
    849443: 'Sketchbook to Style Boot Camp',
    849450: 'Sketchbook to Style Boot Camp',
    926369: 'Sketchbook to Style Boot Camp',
    949995: 'Emails'}

In [ ]:
# if the delta is less than 10 days, then update the unknown with the form
for k,v in sub_forms.items():
    if (v['dt']- v['sub_create']).total_seconds()/60/60/24 < 10:
        values = [forms_key[v['form']], k]
        cursor.e('UPDATE subscriber_data SET `lead_opt-in`=%s WHERE id=%s', values=values)
        print(values, v['form'])

In [ ]:
conn.commit()

##### Using MailChimp

In [ ]:
mailchimp_subs = [a for a in all_rds_list if a['mailchimp_sub_date'] and ((not a['lead_source'] or a['lead_source']=='Unknown') or (not a['lead_opt-in'] or a['lead_opt-in']=='Unknown'))]
len(mailchimp_subs), next(iter(mailchimp_subs), None)
nones = ['None', None, 'null', 'Unknown']
for i, sub in enumerate(mailchimp_subs):
    print(f'Updating # {i}/{len(mailchimp_subs)}', end='\t\r')
    mc_data = {}
    if sub['id'] < 1000000: continue
    if sub['lead_source'] in nones: mc_data['lead_source'] = 'MailChimp'
    if sub['lead_opt-in'] in nones: mc_data['lead_opt-in'] = 'MailChimp'
    try:
        clear_output()
        print(mc_data)
        print(sub)
        input('continue?')
        update_sub_everywhere_with_misc(sub['id'], cursor, session=session, **mc_data)

    except KeyboardInterrupt as k:
        break
    except Exception as e:
        pass

In [ ]:
cursor.e('UPDATE subscriber_data SET `lead_opt-in`="MailChimp" WHERE `lead_opt-in`="Unknown" and `lead_source`="MailChimp"')

In [ ]:
conn.commit()

##### Other

In [ ]:
test = [c['created_at'] for c in cursor.e('SELECT created_at FROM subscriber_data WHERE `lead_opt-in`="Unknown" AND state="active"')]

## Analysis

### Gather data from databases

In [ ]:
cursor.e('SELECT * FROM subscriber_data LIMIT 1')

In [ ]:
d = lambda pur: datetime.strptime(pur, '%Y-%m-%d') # Date conversion

subs = cursor.e("SELECT * FROM subscriber_data") # Get all subscribers
purchases = cursor.e("SELECT * FROM purchase_data") # Get all purchases
purchases = [{'purchase_date': d(p['purchase_date']) if p['purchase_date'] else None, **p} for p in purchases]
launches = cursor.e("SELECT * FROM launch_data")
locs = [[s['id'], (float(s['loc_lat']), float(s['loc_lng']))] for s in subs if s['loc_lat'] and s['loc_lng']]
ids, coords = zip(*locs)
countries = [iso3166.countries_by_alpha2.get(r['cc'], None).name for r in rg.search(coords, mode=1)]; len(countries)
country_dict = {a[0]:a[1] for a in zip(ids,countries)} #dictionary of countries for subscriber id
subs = [{**s, 'country': country_dict.get(s['id'], None)} for s in subs]
sub_dict = {s['id']:s for s in subs}; len(countries) # Dictionary by id

### One-time Data clean up

Replace low subscriber_id entries (make sure purchases are loaded)

In [ ]:
old_sub_ids = [p['subscriber_id'] for p in purchases if not sub_dict[p['subscriber_id']]['country']]; old_sub_ids

In [ ]:
for old_sub_id in old_sub_ids:
    email_address = [p for p in purchases if p['subscriber_id']==old_sub_id][0]['email_address']; email_address
        # Find the real id
    try:
        sub_data = get_single_subscriber(email_address=email_address, cancelled=True); sub_data
        if sub_data['id'] == old_sub_id: continue # This one doesn't need an update

        #     Create the real subscriber_data entry    
        update_single_subscriber(sub_data['id'], conn)
            # Update the purchase_data table with the new reference
        sub_transition = [sub_data['id'], old_sub_id]; sub_transition
        sql_stmt = 'UPDATE purchase_data SET subscriber_id=%s WHERE subscriber_id=%s'
        updated_rows = cursor.execute(sql_stmt, sub_transition); updated_rows
        if updated_rows == 0: raise Exception('no update')
            # Delete the old subscriber_data entry
        updated_rows = cursor.execute(f'DELETE from subscriber_data WHERE id={old_sub_id}'); updated_rows
        if updated_rows == 0: raise Exception('no delete')
            # verify no purchase data issues
    except Exception as e:
        print(old_sub_id, e)
        continue

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE id=4603')

In [ ]:
conn.commit()

In [ ]:
# Try to do the update via the page
for t in temp:
    if t['subscriber_id']>20000:
        print(update_single_subscriber(t['subscriber_id'], conn))

In [ ]:
manually_add_location(814, 'San Francisco', 'California', conn=conn)

Fix subscribers with no subscription dates

In [ ]:
cursor.execute('SELECT * FROM subscriber_data WHERE subscribe_date IS NULL AND state != "unknown"'); data=cursor.fetchall()
len(data)

In [ ]:
conv = lambda x: datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z').strftime('%Y-%m-%d')
values = [[conv(d['created_at']), d['id']] for d in data]; values

In [ ]:
cursor.executemany('UPDATE subscriber_data SET subscribe_date=%s WHERE id=%s', values)

In [ ]:
conn.commit()

### Audience

Create FB audience from subscriber file

In [ ]:
subscribers[0]

In [ ]:
header = ['email', 'fn', 'country', 'value']

In [ ]:
data= [[s['email_address'], s['first_name'], s['country'], s['purchase']] for s in subscribers]

In [ ]:
df = pd.DataFrame(data, columns=header)
df.to_csv('/Users/nish/Downloads/subscribers.csv', index=False)

### Purchase / Launch Analysis

In [ ]:
def include(item, _all=True, **attrs):
    """ 
    Parameters
    ==========
    item (dict): is purchase or launch, must have the "subscriber_id"
    attr (kwarg): the aspect to evaluate
    _all (bool): If true, use "all", otherwise use "any"
    """
    bool_fn = all if _all else any
        
    attrs = {k:v if type(v) in [list, tuple] else [v] for k,v in attrs.items()}; # ensure they are all lists
    
    subscriber = subscriber_dict.get(item.get('subscriber_id', {}), {}) # Get the subscriber info
    
    return bool_fn(subscriber.get(k, object) in v for k,v in attrs.items())

In [ ]:
log_level('i')

In [ ]:
a = cursor.e('SELECT * FROM subscriber_data WHERE `lead_opt-in`="Style Mini-course"'); a
len(a)

In [ ]:
len([b for b in a if b['state']=="cancelled"])

In [ ]:
attrs = {}
# attrs['country'] = ['United States of America', 'Canada']
# attrs['lead_opt-in'] = ['Influence Map', 'Style Mini-course']
# attrs['lead_opt-in'] = 'Influence Map'
attrs['lead_opt-in'] = 'Style Mini-course'
_all = True; join_str = " AND " if _all else " OR "


logging.info(f'Evaluating: {join_str.join([f"({k} IN {v})" for k,v in attrs.items()])}')
temp_purchases = [p for p in purchases if include(p, _all=_all, **attrs)]
temp_launches = [l for l in launches if include(l, _all=_all, **attrs)]
f'purchase rate is {len(temp_purchases)}/{len(temp_launches)} {round(len(temp_purchases)/len(temp_launches)*100,2)}%'

#### % buy by audience type

In [ ]:
# Number of launches to americans/canadians
sql_stmt = f"""\
SELECT * FROM launch_data 
WHERE launch="Launch_6"
"""
cursor.execute(sql_stmt); temp = cursor.fetchall(); len(temp), temp[:2]
countries = ['United States of America', 'Canada']

loc_launches = [t for t in temp if subscriber_dict[t['subscriber_id']]['country'] in countries]; len(loc_launches)
# other_countries = [subscriber_dict[t['id']]['country'] for t in temp if subscriber_dict[t['id']]['country'] not in countries]; other_countries
# Counter(other_countries)
# purchases = len(temp)

In [ ]:
# People who went through the launch
# [[l['id'], subscriber_dict[l['id']]['email_address']] for l in loc_launches]

In [ ]:
# Number of purchases by americans/canadians
sql_stmt = f"""\
SELECT * FROM purchase_data 
WHERE launch="Launch_6"
"""
cursor.execute(sql_stmt); temp = cursor.fetchall()
countries = ['United States of America', 'Canada']

loc_purchases = [t for t in temp if subscriber_dict.get(t.get('subscriber_id', None), {}).get('country', None) in countries]; len(loc_purchases)
# purchases = len(temp)

In [ ]:
f'US & CA purchase rate is {round(len(loc_purchases)/len(loc_launches)*100,2)}%'

In [ ]:
# purchase countries
Counter([subscriber_dict.get(t.get('subscriber_id', None), {}).get('country', None) for t in temp])

#### %buy by misc

#### % buy by length subscribed

In [ ]:
# % buy by # of emails received

In [ ]:
# % buy of north americans that go through prelaunch nurturing

In [ ]:
cursor.quick_info('launch_data', launch='Evergreen_2')

#### Day breakdown of Launches Given

In [ ]:
logging.getLogger().setLevel(logging.WARN)
sub_end_datetimes = get_past_launch_completions(conn=conn, include_unsubscribers=True) # Get the end dates for launches


In [ ]:
sub_end_dates = {k:datetime.strftime(v,"%Y-%m-%d") for k,v in sub_end_datetimes.items()}
list(sub_end_dates.items())[:3]

In [ ]:
na_end_dates = {k:v for k,v in sub_end_dates.items() if k in sub_dict and sub_dict[k]['country'] in countries}; 
Counter(countries).most_common(3), len(na_end_dates), list(na_end_dates.items())[:3]

In [ ]:
Counter(na_end_dates.values()).most_common()

#### Day breakdown of Upcoming Launches

In [ ]:
upcoming_launch_completions = get_upcoming_launch_completions(conn=conn)

In [ ]:
up_launch_dates = {k:datetime.strftime(v,"%Y-%m-%d") for k,v in upcoming_launch_completions.items()}
list(up_launch_dates.items())[:3]

In [ ]:
na_up_launch_dates = {k:v for k,v in up_launch_dates.items() if k in sub_dict and sub_dict[k]['country'] in countries}; 
Counter(countries).most_common(3), len(na_up_launch_dates), list(na_up_launch_dates.items())[:3]


In [ ]:
Counter(na_up_launch_dates.values()).most_common(5)

### Ad Analysis

In [ ]:
# Find out the number of subscribers, inactives per day for all.
# Then look at how long it takes inactives to subscribe
# Then look at the breakdown by ad type

#NOW() - INTERVAL 5 HOUR 
sql_stmt = """\
SELECT *, STR_TO_DATE(created_at, "%Y-%m-%dT%T.%fZ") AS d 
FROM subscriber_data 
HAVING d > "2019-06-11"
"""
cursor.execute(sql_stmt)
data = cursor.fetchall(); len(data)

In [ ]:
preactives = [d for d in data if d['state'] == 'inactive']
actives = [d for d in data if d['state'] == 'active']
postactives = [d for d in data if not d['state'] in  ['active', 'inactive']]
len(preactives), len(actives), len(postactives)

In [ ]:
def created_at_date(x): 
    return datetime.strptime(x, '%Y-%m-%dT%H:%M:%S.000Z').date()

def created_at_on_day(created_at, day):
    return created_at_date(created_at) >= day and created_at_date(created_at) < day + timedelta(days=1)

created_at_list = []
date_list = [date.today() - timedelta(days=x) for x in range(0, 31)]; date_list

for date_item in date_list[::-1]:
    lp_convs = [d for d in data if created_at_on_day(d['created_at'], date_item)]
    fb_ad_convs = [l for l in lp_convs if l['lead_source'] == "FB Ad"]
    fb_ad_spend = 70
    infl_convs = [l for l in lp_convs if l['lead_opt-in'] == "Influence Map"]
    infl_spend = 50
    mini_course_spend = 20
    mini_course_convs = [l for l in lp_convs if l['lead_opt-in'] == "Style Mini-course"]
    preactives = [l for l in lp_convs if l['state'] == 'inactive']
    actives = [l for l in lp_convs if l['state'] == 'active']
    postactives = [l for l in lp_convs if not l['state'] in ['active', 'inactive']]
    
#     influence_map_conversions = [l for l in lp_convs if l[]]
    
    created_at_list.append(
        {
            "date": date_item,#datetime.strftime(date_item, '%d'), 
            'lp_conv': len(lp_convs), 
            'unconf': len(preactives),
            'active': len(actives),
            'unsub': len(postactives),
            'unconf%': round(len(preactives)/len(lp_convs)*100,0),
            'active%': round(len(actives)/len(lp_convs)*100,0),
            'unsub%': round(len(postactives)/len(lp_convs)*100,0),
            'fb_ad': len(fb_ad_convs),
            'infl': len(infl_convs),
            'mini': len(mini_course_convs),
            'fb_ad_$': fb_ad_spend,
            'infl_$': infl_spend,
            'mini_$': mini_course_spend,
            'fb_ad_$/fb': round(fb_ad_spend/max(1,len(actives)),2),
            'infl_$/infl': round(infl_spend/max(1,len(infl_convs)),2),
            'mini_$/mini': round(mini_course_spend/max(1,len(mini_course_convs)),2),
        }
    )

In [ ]:
lp_convs[0]

In [ ]:
# print(json.dumps(created_at_list, indent=2))
pd.DataFrame(created_at_list).set_index('date')

In [ ]:
# See if the number of actives increases over time

In [ ]:
subscribers, date_item

In [ ]:
created_at_date(data[0]['created_at']) < date_list[0]

In [ ]:
# Rows of days, count of preactives, actives, postactives
array = {i: [d for d in data if created_at_date(d['created_at']) > i and ] for i in date_list}
array

## Continuous Updates

In [ ]:
# Update emails

In [ ]:
# All users without some custom field
ck_sub_data = get_all_sub_data(columns=None)

In [ ]:
# People that don't have 
ck_sub_data[1]

## Other

In [ ]:
# Style Tag issues
# People that have style original and style evg
# People with style completed, but no style emails
style_comp = get_api_data('https://api.convertkit.com/v3/tags/451997/subscriptions', test=False, session=session)
style_comp_ids = [a['subscriber']['id'] for a in style_comp]; len(style_comp_ids), style_comp_ids[:3]
len(style_comp)

In [ ]:
# Get the people with no style emails
email_ids = []
email_ids += [a['id'] for a in get_seq_dict(383238)['course']['email_templates']]
email_ids += [a['id'] for a in get_seq_dict(371104)['course']['email_templates']]
email_ids
# cursor.execute()

In [ ]:
cursor.quick_info('email_data')

In [ ]:
cursor.execute('SELECT DISTINCT subscriber_id FROM email_data GROUP BY subscriber_id WHERE subscriber_id')

In [ ]:
# Troubleshooting
get_sequence_end_date(a)

In [ ]:
json.loads(result.text)['message'][:24]

In [ ]:
# Create API Call for subscriptions that were missed
for missing_id in rds_missing_subs[4:]:
    url = 'https://td9rz0ja7b.execute-api.us-east-1.amazonaws.com/prod/subscription'
    data = json.dumps({"subscriber": { "id": missing_id } })
    headers = {'content-type': 'application/json'}
    result = session.post(url, data=data, headers=headers)
    if result.status_code==200:
        result_json = json.loads(result.text)
        if 'executionArn' in result_json:
            print(result.status_code, result_json.get('executionArn')[-19:])
        else:
            print(result.status_code, missing_id, result_json['message'][:24])
    else:
        print(result.status_code, missing_id)
    time.sleep(5)

In [ ]:
# cursor.quick_info('email_data')
# cursor.execute('SHOW KEYS FROM email_data;'); 
cursor.execute('SHOW INDEX FROM email_data;'); 
cursor.fetchall()

In [ ]:
from urllib.parse import urlencode
urlencode({'a':'a a a a', 'b':'Bacon!.'})
urlencode({})

#### No emails?

In [ ]:
# 'SELECT STR_TO_DATE(created_at, "%Y-%m-%dT%H:%i:%s") AS d FROM subscriber_data HAVING d >= NOW() - INTERVAL 2 HOUR ORDER BY d DESC

sql_stmt = """\
SELECT e.subscriber_id, s.*, Count(*) as count 
FROM email_data e
LEFT JOIN subscriber_data s
    ON e.subscriber_id = s.id
GROUP BY subscriber_id 
HAVING count < 5 
LIMIT 1
"""
cursor.execute(sql_stmt)
# cursor.execute('SELECT * FROM email_data WHERE subscriber_id = 137074446')
cursor.fetchall()

#### Sequence End Date Testing

See that if the create date is None or a long time ago, but no emails exist, that the completion date will assume emails start going today. Maybe it can notify when people aren't getting the emails

In [ ]:
nurturing_seq_dict = get_seq_dict(383238, session=session); nurturing_seq_dict['course']['name'] # 375580

In [ ]:
nurturing_subs = get_all_seq_sub_data(383238, session=session) 

In [ ]:
# sub_id, create_date = nurturing_subs[0]; 
sub_id = 492139769 #498092029
create_date = '2019-06-11T22:02:51.000Z' # '2019-05-30T23:02:51.000Z'
email_dict = {}
email_dict[sub_id] = scrape_email_list(sub_id, ck_app_session=None); email_dict

In [ ]:
logging.getLogger().setLevel(logging.DEBUG)
end_date = get_sequence_end_date(nurturing_seq_dict, email_dict[sub_id], create_date=create_date, sub_id=sub_id); end_date


In [ ]:
get_sequence_end_date(nurturing_seq_dict, a)

In [ ]:
nurturing_seq_dict#; email_dict[sub_id]

#### Other

In [ ]:
cursor.quick_info()

In [ ]:
# cursor.execute('SELECT DISTINCT state FROM subscriber_data')
cursor.execute('SELECT * FROM subscriber_data WHERE lead_source = "None"')
cursor.fetchall()
# ids = [i['id'] for i in cursor.fetchall()]

In [ ]:
url = f'https://api.convertkit.com/v3/custom_fields'
data = json.dumps({'api_key': os.environ['CK_API_KEY']})
headers = {'content-type': 'application/json'}
r = requests.get(url=url, data=data, headers=headers)
# if r.status_code != 200: raise Exception
print(r.text)

In [ ]:
a = {'a': 1}
b = {'b': 2}
a.update(b); a

In [ ]:
try:
    try:
        sub = get_single_subscriber(email_address="dnishiyama+test2@gmail.com")
    except Exception as e:
        print('uh oh')
except Exception as e:
    print('got exception', e)
if 'email_address' in sub and sub['email_address']:
    print('sub')
sub

In [ ]:
(datetime.now() - datetime.strptime("2019-04-08T19:12:24.000Z", "%Y-%m-%dT%H:%M:%S.%fZ")) < timedelta(days=1)

In [ ]:
d = datetime.strptime('2018-02-21', '%Y-%m-%d') - datetime.strptime('2018-02-20', '%Y-%m-%d'); d
timedelta(days=1).total_seconds()/60/60/24

In [ ]:
# Average email subscription duration before purchase
# Get the list of purchasers
sql_stmt = """
SELECT s.id, p.email_address, p.purchase_date, s.subscribe_date
FROM purchase_data p
INNER JOIN subscriber_data s ON p.subscriber_id = s.id"""
cursor.execute(sql_stmt); purchasers = cursor.fetchall(); purchasers
# Subtract their subscription date
delay_days = lambda pur, sub: (datetime.strptime(pur, '%Y-%m-%d') - datetime.strptime(sub, '%Y-%m-%d')).total_seconds()/60/60/24
purchasers = [[*e.values()] for e in purchasers if e['purchase_date'] and e['subscribe_date']] #remove nones
purchase_delay = [[*e, delay_days(e[2], e[3])] for e in purchasers if delay_days(e[2], e[3]) > 0]; purchase_delay[0]

In [ ]:
# Graph that
plt.hist([e[4] for e in purchase_delay], bins=50)